## Погнали тренить GPT

<details>
  <summary>Вспомним, как работает трансформер 🤖</summary>

В оригинальной модели Трансформера, описанной в статье «Внимание — это все, что вам нужно», архитектура разделена на две основные части: кодер и декодер. Обе части состоят из слоев, имеющих одинаковую общую структуру, но служащих разным целям.

На рисунке ниже, изображена архитектура модели Transformer. Она состоит из двух основных частей: **кодера (encoder)** и **декодера (decoder)**.

![Figure_1](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-04/assets/Figure_1.png)

### Кодер (Encoder)
Кодер обычно находится в левой части архитектуры. Он состоит из нескольких слоев, каждый из которых включает:
1. **Multi-Head Attention** — механизм внимания, который позволяет модели фокусироваться на разных частях входных данных.
2. **Add & Norm** — слой, который добавляет входные данные к результату внимания (residual connection) и применяет нормализацию.
3. **Feed Forward** — полносвязный слой, который применяется к каждому элементу последовательности независимо.
4. **Add & Norm** — снова добавляет входные данные к результату и нормализует.

Эти слои повторяются несколько раз (обычно 6 или более) для создания глубокой модели.

### Декодер (Decoder)
Декодер обычно находится в правой части архитектуры. Он также состоит из нескольких слоев, но имеет дополнительные компоненты:
1. **Masked Multi-Head Attention** — механизм внимания, который маскирует будущие токены, чтобы предотвратить "подглядывание" вперед.
2. **Add & Norm** — слой, который добавляет входные данные к результату внимания и нормализует.
3. **Multi-Head Attention** — механизм внимания, который учитывает выход кодера.
4. **Add & Norm** — снова добавляет входные данные к результату и нормализует.
5. **Feed Forward** — полносвязный слой, аналогичный тому, что используется в кодировщике.
6. **Add & Norm** — завершающий слой добавления и нормализации.

### Входы и выходы
- **Input Embedding** и **Positional Encoding** относятся к входным данным, которые подаются в кодер.
- **Output Embedding** и **Outputs (shifted right)** относятся к выходным данным, которые обрабатываются декодером.

### **Кодер**:

#### Роли:
Роль кодировщика заключается в обработке входных данных и создании представления, отражающего отношения между элементами (например, словами в предложении). Эта часть преобразователя не генерирует никакого нового контента, она просто преобразует входные данные в состояние, которое может использовать декодер.

#### Функциональность системы:
Каждый уровень кодера имеет механизмы самовнимания и нейронные сети прямой связи. Механизм самовнимания позволяет каждой позиции кодера обрабатывать все позиции предыдущего уровня кодера — таким образом, он может изучать контекст вокруг каждого слова.

#### Контекстные встраивания:
Выходные данные кодера представляют собой серию векторов, которые представляют входную последовательность в многомерном пространстве. Эти векторы часто называют контекстными вложениями, поскольку они кодируют не только отдельные слова, но и их контекст внутри предложения.

#### Математическое описание:

1. **Самовнимание (Self-Attention)**:
   - **Входной вектор $X$**: Это матрица, представляющая входную последовательность (например, слова в предложении) в виде эмбеддингов.
   - **Ключи $K$**, **запросы $Q$** и **значения $V$**: Эти матрицы получаются путем умножения входного вектора $X$ на весовые матрицы $W_K$, $W_Q$ и $W_V$ соответственно:

     $$
     Q = XW_Q, \quad K = XW_K, \quad V = XW_V
     $$

   - **Внимание $A$**: Вычисляется путем скалярного произведения запросов и ключей, нормированного на размерность ключей $d_k$:
   
     $$
     A = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)
     $$

   - **Выход самовнимания $Z$**: Это результат умножения матрицы внимания на значения:

     $$
     Z = AV
     $$

```python
import numpy as np

def self_attention(X, d_k):
    """
    Description:
      Реализует механизм Self-Attention.

    Args:
        X (numpy.ndarray): Входной вектор (матрица эмбеддингов), размерностью (n_samples, embedding_dim).
        d_k (int): Размерность ключей.

    Returns:
        numpy.ndarray: Выход самовнимания Z.
    """
    # Инициализация весовых матриц случайными значениями
    embedding_dim = X.shape[1]
    W_Q = np.random.rand(embedding_dim, d_k)
    W_K = np.random.rand(embedding_dim, d_k)
    W_V = np.random.rand(embedding_dim, d_k)

    # Вычисление матриц запросов, ключей и значений
    Q = X @ W_Q
    K = X @ W_K
    V = X @ W_V

    # Вычисление матрицы внимания
    attention_scores = Q @ K.T
    scaled_attention_scores = attention_scores / np.sqrt(d_k)

    # Применение функции softmax для получения весов внимания
    attention_weights = np.exp(scaled_attention_scores) / np.sum(np.exp(scaled_attention_scores), axis=-1, keepdims=True)

    # Вычисление выходного вектора
    Z = attention_weights @ V

    return Z

# Пример использования
# Предположим, у нас есть последовательность из 3 слов, где каждое слово представлено вектором размерности 4
X = np.array([[1.0, 0.5, 0.2, 0.8],
              [0.3, 0.9, 0.1, 0.4],
              [0.6, 0.2, 0.7, 0.5]])

d_k = 2  # Пример размерности ключей

# Получение выходного вектора самовнимания
output_Z = self_attention(X, d_k)

print("Входной вектор X:\n", X)
print("\nВыход самовнимания Z:\n", output_Z)
```

2. **Нейронная сеть прямой связи (Feedforward Neural Network)**:
   - Выход самовнимания $Z$ проходит через два полносвязных слоя с функцией активации (например, ReLU) между ними:

     $$
     \text{FFN}(Z) = \max(0, ZW_1 + b_1)W_2 + b_2
     $$

   - **$W_1$, $W_2$**: Весовые матрицы первого и второго полносвязного слоя.
   - **$b_1$, $b_2$**: Смещения первого и второго полносвязного слоя.

#### **Весовые матрицы $W_K$, $W_Q$ и $W_V$ в контексте трансформеров**

1. Общий контекст

  В архитектуре трансформера важнейшую роль играет механизм внимания, который позволяет модели учитывать контекст при обработке каждого элемента последовательности (например, слов в предложении). Для этого механизм внимания использует три ключевых компонента: **запросы (queries)**, **ключи (keys)** и **значения (values)**. Эти компоненты формируются с помощью весовых матриц $W_Q$, $W_K$ и $W_V$ соответственно.

2. Формирование весовых матриц

  2.1 Входные данные.
  
  Допустим, у нас есть входная последовательность $X$ размером $n \times d_{model}$, где:
  - $n$ — количество токенов в последовательности (длина предложения).
  - $d_{model}$ — размерность эмбеддингов (например, 512 или 1024).

  Эта последовательность $X$ поступает на вход трансформеру, где сначала проходит через линейные слои, которые преобразуют её в запросы $Q$, ключи $K$ и значения $V$.

  2.2 Линейные преобразования.

  Для создания запросов, ключей и значений, входная последовательность умножается на три разные весовые матрицы $W_Q$, $W_K$ и $W_V$:

  $$
  Q = XW_Q, \quad K = XW_K, \quad V = XW_V
  $$

  где:
  - $W_Q$ — весовая матрица для запросов размера $d_{model} \times d_k$.
  - $W_K$ — весовая матрица для ключей размера $d_{model} \times d_k$.
  - $W_V$ — весовая матрица для значений размера $d_{model} \times d_v$.

  Размерность $d_k$ и $d_v$ может быть выбрана по-разному, но обычно $d_k = d_v = d_{model}$. Эти весовые матрицы обучаются в процессе тренировки модели и их задача — найти такие представления запросов, ключей и значений, которые позволяют оптимально учитывать контекст.

3. Почему используются разные матрицы?

  3.1 Запросы ($W_Q$)

  Матрица $W_Q$ преобразует входные векторы в запросы, которые используются для оценки важности других элементов последовательности относительно данного. Это означает, что каждый запрос пытается "выяснить", на какие другие слова в предложении ему следует обратить внимание.

  3.2 Ключи ($W_K$)

  Матрица $W_K$ преобразует входные векторы в ключи. Ключи используются для сопоставления с запросами. По сути, ключи содержат информацию о том, "насколько важен" каждый элемент последовательности при сопоставлении с запросом. Чем ближе запрос к ключу (по мере скалярного произведения), тем большее внимание будет уделено соответствующему элементу.

  3.3 Значения ($W_V$)

  Матрица $W_V$ преобразует входные векторы в значения. Значения передают фактическую информацию, которая будет использована при вычислении окончательного представления после применения механизма внимания. Итоговое значение для каждого токена будет взвешенной суммой значений всех токенов, где веса определяются степенью внимания (вычисленной через запросы и ключи).

4. Механизм внимания

  4.1 Вычисление внимания

  Запросы $Q$ и ключи $K$ используются для вычисления матрицы внимания. Для этого рассчитывается скалярное произведение запросов и ключей с последующей нормализацией по размерности $d_k$ и применением softmax:

  $$
  A = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)
  $$

  Где $A$ — матрица внимания, показывающая, какие элементы последовательности оказывают наибольшее влияние друг на друга.

  4.2 Применение внимания к значениям

  После получения матрицы внимания, её умножают на матрицу значений $V$, чтобы получить финальное представление каждого токена, учитывающее контекст:

  $$
  Z = AV
  $$

5. Заключение

  Весовые матрицы $W_Q$, $W_K$ и $W_V$ служат для преобразования входных данных в запросы, ключи и значения, которые необходимы для работы механизма внимания. Эти матрицы обучаются в процессе тренировки модели и играют решающую роль в определении того, как модель интерпретирует контекст каждого слова в предложении.

  Такое внимание к контексту позволяет трансформерам эффективно решать задачи обработки естественного языка, обеспечивая высокую производительность в задачах перевода, понимания текста и многих других.

## **От входа к выходу внутри кодера**

### 1. **Входные данные:**

  - **Входная последовательность $X$**: Матрица размерности $(N, L, D_{model})$, 

  где:

  **Batch Size ($N$) - Размер пакета:**

  * Batch Size определяет, сколько **независимых** последовательностей обрабатывается моделью **одновременно** за один полный проход обучения или инференса (предсказания).

  **Длина последовательности ($L$) - Length of the sequence:**

  * Длина последовательности определяет количество **элементов** (токенов) в каждой **отдельной** последовательности внутри пакета.

  **Размерность эмбеддингов ($D_{model}$) - Dimension of the embeddings:**

  * Фиксированное векторное представление какого-либо объекта (например, слова, токена).

  **Как получается матрица X:**

  1. **Эмбеддинг элементов:**  Каждый элемент в последовательности (например, слово в предложении) преобразуется в эмбеддинг - вектор размерности $D_{model}$.

  2. **Формирование последовательности:** Для каждой последовательности в пакете мы получаем матрицу размерности $(L, D_{model})$. Каждая строка этой матрицы соответствует эмбеддингу одного элемента последовательности. Таким образом, у нас есть $L$ строк (по количеству элементов в последовательности) и $D_{model}$ столбцов (по размерности эмбеддинга).

  3. **Формирование пакета:**  Поскольку у нас есть $N$ независимых последовательностей в пакете, мы "складываем" эти матрицы $(L, D_{model})$ друг на друга. Это создает трехмерный тензор (или матрицу размерности 3) $X$ с размерами $(N, L, D_{model})$.

### **В итоге:**

Матрица $X$ размерности $(N, L, D_{model})$ представляет собой трехмерный тензор. Если рассматривать его как набор двумерных матриц, то он состоит из $N$ матриц размерности $(L, D_{model})$, "наложенных" друг на друга. Каждая из этих внутренних матриц (соответствующая одной последовательности в пакете) имеет $L$ строк (по количеству элементов в последовательности) и $D_{model}$ столбцов (по размерности эмбеддингов).

### **Пример:**

```Python
import torch

# Пример данных: 3 предложения в батче
sentences = [
    "Всем привет! Я увлекаюсь искусственным интеллектом",  # 7 токенов
    "Привет, как дела?",                                   # 4 токена
    "ИИ — это интересно!"                                  # 5 токенов
]

# 1. Упрощенная токенизация для каждого предложения
batch_tokens = [
    ["[CLS]", "Всем", "привет", "!", "Я", "увлекаюсь", "искусственным", "интеллектом", "[SEP]"],  # L=9
    ["[CLS]", "Привет", ",", "как", "дела", "?", "[SEP]", "[PAD]", "[PAD]"],                      # L=9 (с паддингом)
    ["[CLS]", "ИИ", "—", "это", "интересно", "!", "[SEP]", "[PAD]", "[PAD]"]                      # L=9 (с паддингом)
]

N = len(sentences)  # Размер батча = 3
L = 9               # Максимальная длина последовательности (после паддинга)
D_model = 512       # Размерность эмбеддингов

# 2. Создаем случайные эмбеддинги для всего батча
embeddings = torch.randn(N, L, D_model)

# 3. Формируем входной тензор X
X = embeddings

print("Форма тензора X:", X.shape)  # (3, 9, 512)
print("\nСтруктура данных:")
print(f"• Количество батчей (N): {N}")
print(f"• Длина последовательности (L): {L} (с учетом паддинга)")
print(f"• Размерность эмбеддингов (D_model): {D_model}\n")

# ================= Визуализация процесса =================
# Визуализация содержимого
for batch_idx in range(N):
    print(f"Батч {batch_idx + 1} ({sentences[batch_idx][:20]}...):")
    print(f"Токены: {batch_tokens[batch_idx]}")
    
    # Показываем первые 3 элемента эмбеддингов для ключевых позиций
    print("\nПримеры эмбеддингов:")
    print(f"• [CLS] токен: {X[batch_idx, 0, :3].detach().numpy().round(4)}...")
    print(f"• 3-й токен:   {X[batch_idx, 2, :3].detach().numpy().round(4)}...")
    print(f"• [SEP] токен: {X[batch_idx, -3, :3].detach().numpy().round(4)}...")
    print(f"• [PAD] токен: {X[batch_idx, -1, :3].detach().numpy().round(4)}...")
    print("-" * 60)
```

```Python
# ================= Подробный вывод для первого батча =================
# Визуализация содержимого тензора X
for batch_idx in range(N):
    print(f"Батч {batch_idx + 1} ({sentences[batch_idx][:20]}...):")
    print(f"Токены: {batch_tokens[batch_idx]}")
    
    # Выводим все эмбеддинги для текущей последовательности
    print("\nЭмбеддинги:")
    for token_idx in range(L):
        print(f"• Токен {token_idx}: {X[batch_idx, token_idx].detach().numpy().round(4)}...")
    print("-" * 60)
```

### 2. **Позиционные кодирования (Positional Encodings):**

  **Зачем вообще нужны позиционные кодирования?** Традиционные рекуррентные нейронные сети (RNN), такие как LSTM или GRU, обрабатывают последовательность токен за токеном, и порядок обработки естественным образом учитывает положение токена. Однако, архитектуры, основанные на механизме внимания, такие как Transformer, обрабатывают все токены последовательности параллельно. Из-за этого они теряют информацию о порядке следования токенов. Позиционные кодирования вводятся для того, чтобы "подсказать" модели, где находится каждый токен в последовательности.

  Когда обученная модель получает на вход предложение для анализа (например, для классификации или машинного перевода), кодер Transformer также обрабатывает все токены входной последовательности параллельно, формируя контекстуализированные представления для каждого токена.

  Так же важно отметить, что позиционные кодирования ($PE$) имеют ту же размерность, что и эмбеддинги слов ($D_{model}$). Это ключевой момент, поскольку позволяет складывать их вместе поэлементно.

  ![Figure_2](https://raw.githubusercontent.com/Verbasik/Weekly-arXiv-ML-AI-Research-Review/refs/heads/develop/2025/week-04/assets/Figure_2.png)

  - Для учета порядка слов в последовательности к входным эмбеддингам добавляются позиционные кодирования $PE$ той же размерности $D_{model}$.
  - $PE$ вычисляются по следующим формулам:
    $$
    PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/D_{model}}}\right) \\
    PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/D_{model}}}\right)
    $$
    
    где:

  *   **$pos$ (позиция):**
      *   Представляет собой целое число, указывающее на позицию токена в последовательности.
      *   Нумерация начинается с 0. Первый токен имеет $pos = 0$, второй - $pos = 1$, и так далее.
      *   Например, в предложении "Собака лает громко", "Собака" имеет $pos = 0$, "лает" - $pos = 1$, "громко" - $pos = 2$.
  *   **$i$ (индекс измерения):**
      *   Это индекс, который определяет конкретное измерение в векторе позиционного кодирования.
      *   $i$ принимает значения от 0 до $D_{model}/2 - 1$.
      *   Для каждого значения $i$ вычисляется пара значений: одно с синусом, другое с косинусом.
      *   Например, если $D_{model} = 512$, то $i$ будет принимать значения от 0 до 255.
  *   **$D_{model}$ (размерность модели):**
      *   Это размерность эмбеддингов слов и позиционных кодирований.
      *   Обычно это значение равно 512, но может быть и другим.
      *   $D_{model}$ определяет длину вектора позиционного кодирования.

  **Почему используются синус и косинус?**

  *   **Уникальность:** Синус и косинус позволяют генерировать уникальные позиционные кодирования для каждой позиции.
  *   **Периодичность:** Периодичность этих функций позволяет модели легко различать относительные позиции токенов.
  *   **Экстраполяция:** Модель может экстраполировать на более длинные последовательности, чем те, на которых она была обучена.
  *   **Относительные позиции:** Разница между позиционными кодированиями для соседних позиций остается относительно постоянной, что помогает модели понимать относительное положение токенов.

  **Генерация уникальных позиционных кодирований:**

  Для каждой позиции $pos$ и каждого индекса измерения $i$ относительно трехмерного тензора $X$ размерности $(N, L, D_{model})$ вычисляется уникальное значение. Поскольку $i$ пробегает значения от 0 до $D_{model}/2 - 1$, для каждой позиции $pos$ получается вектор размерности $D_{model}$. Первые $D_{model}/2$ элементов этого вектора вычисляются с помощью синуса, а остальные $D_{model}/2$ - с помощью косинуса.

  **Совместимость размерностей:**

  Позиционные кодирования имеют ту же размерность, что и эмбеддинги слов ($D_{model}$), чтобы их можно было поэлементно сложить. Это позволяет модели учитывать как семантическое значение слова (из эмбеддинга), так и его позицию в последовательности (из позиционного кодирования).

### **В итоге:**

Итоговый вход для первого слоя кодера получается путем поэлементного сложения эмбеддингов слов $X$ и позиционных кодирований $PE$: $X_{embedded} = X + PE$.

**Пример:**

Предположим, что $D_{model} = 4$. Тогда для позиции $pos = 1$ и $i = 0$ мы получим:

$$PE_{(1, 0)} = \sin\left(\frac{1}{10000^{0}}\right) = \sin(1) \approx 0.84$$
$$PE_{(1, 1)} = \cos\left(\frac{1}{10000^{0}}\right) = \cos(1) \approx 0.54$$

Для $i = 1$:

$$PE_{(1, 2)} = \sin\left(\frac{1}{10000^{2/4}}\right) = \sin\left(\frac{1}{100}\right) \approx 0.01$$
$$PE_{(1, 3)} = \cos\left(\frac{1}{10000^{2/4}}\right) = \cos\left(\frac{1}{100}\right) \approx 1$$

Таким образом, вектор позиционного кодирования для позиции 1 будет примерно равен [0.84, 0.54, 0.01, 1].

- Итоговый вход для первого слоя кодера: $X_{embedded} = X + PE$.

```Python
import torch
import math

def positional_encoding(max_len: int, d_model: int) -> torch.Tensor:
    """
    Description:
        Генерация позиционных кодирований по формуле из оригинальной статьи Transformer.

    Args:
        max_len: Максимальная длина последовательности.
        d_model: Размерность модели (количество признаков).

    Returns:
        Тензор позиционных кодирований формы (max_len, d_model).

    Examples:
        >>> pe = positional_encoding(10, 512)
        >>> pe.shape
        torch.Size([10, 512])
    """
    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
    
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe

def print_embeddings(tensor: torch.Tensor, tokens: list, title: str, max_elements: int = 3) -> None:
    """
    Description:
        Визуализация эмбеддингов с метками токенов.

    Args:
        tensor: Тензор эмбеддингов.
        tokens: Список токенов для визуализации.
        title: Заголовок для вывода.
        max_elements: Максимальное количество элементов для отображения.

    Returns:
        None

    Examples:
        >>> embeddings = torch.randn(5, 512)
        >>> tokens = ["token1", "token2", "token3", "token4", "token5"]
        >>> print_embeddings(embeddings, tokens, "Пример эмбеддингов")
    """
    print(f"\n{title}:")
    for idx, (vec, token) in enumerate(zip(tensor, tokens)):
        elements = vec[:max_elements].detach().numpy().round(4)
        print(f"{idx:2d} {token:15}: [{', '.join(f'{x:7.4f}' for x in elements)}...]")

# Пример данных
sentences = [
    "Всем привет! Я увлекаюсь искусственным интеллектом",
    "Привет, как дела?",
    "ИИ — это интересно!"
]

# Параметры модели
N = len(sentences)  # Размер батча
L = 9               # Максимальная длина последовательности
D_model = 512       # Размерность эмбеддингов

# 1. Токенизация с паддингом
batch_tokens = [
    ["[CLS]", "Всем", "привет", "!", "Я", "увлекаюсь", "искусственным", "интеллектом", "[SEP]"],
    ["[CLS]", "Привет", ",", "как", "дела", "?", "[SEP]", "[PAD]", "[PAD]"],
    ["[CLS]", "ИИ", "—", "это", "интересно", "!", "[SEP]", "[PAD]", "[PAD]"]
]

# 2. Создаем эмбеддинги
embeddings = torch.randn(N, L, D_model)

# 3. Генерируем позиционные кодирования
pe = positional_encoding(L, D_model)

# 4. Комбинируем эмбеддинги с позиционными кодированиями
X_embedded = embeddings + pe  # Broadcasting для батча

# ================= Визуализация процесса =================
print("="*60)
print("Шаг 1: Исходные эмбеддинги токенов")
print(f"Форма тензора: {embeddings.shape}")
for batch_idx in range(N):
    print(f"\nБатч {batch_idx+1}: '{sentences[batch_idx]}'")
    print_embeddings(embeddings[batch_idx], batch_tokens[batch_idx], "Исходные эмбеддинги")

print("\n" + "="*60)
print("Шаг 2: Позиционные кодирования")
print(f"Форма тензора: {pe.shape}")
print_embeddings(pe, [f"Позиция {i}" for i in range(L)], "Примеры кодирований")

print("\n" + "="*60)
print("Шаг 3: Комбинированные эмбеддинги (X + PE)")
print(f"Форма тензора: {X_embedded.shape}")
for batch_idx in range(N):
    print(f"\nБатч {batch_idx+1}: '{sentences[batch_idx]}'")
    print_embeddings(X_embedded[batch_idx], batch_tokens[batch_idx], "Результат сложения")
```

```Python
# ================= Подробный вывод для первого батча =================
print("\n" + "="*60)
print("Детальный анализ первого батча:")
batch_idx = 0

# Исходные данные
print(f"\nТекст: '{sentences[batch_idx]}'")
print(f"Токены: {batch_tokens[batch_idx]}")

# Сравнение для ключевых позиций
for pos in [0, 2, 4, 6, 8]:
    print(f"\nПозиция {pos} ({batch_tokens[batch_idx][pos]}):")
    print(f"Исходный эмбеддинг:  {embeddings[batch_idx, pos, :3].detach().numpy().round(4)}")
    print(f"Позиционное кодир:   {pe[pos, :3].detach().numpy().round(4)}")
    print(f"Комбинированный:     {X_embedded[batch_idx, pos, :3].detach().numpy().round(4)}")
```

### 3. **Многоголовое самовнимание (Multi-Head Self-Attention):**
   - На входе подслоя Multi-Head Attention находится $X_{embedded} = X + PE$.
   - **Линейные проекции:** Входные данные $X_{embedded}$ линейно проецируются в запросы $Q$, ключи $K$ и значения $V$ для каждой головы:
     $$
     Q_i = X_{embedded} W_{Q_i}, \quad K_i = X_{embedded} W_{K_i}, \quad V_i = X_{embedded} W_{V_i}
     $$
     
      где:
      
      - $W_{Q_i} \in \mathbb{R}^{D_{model} \times D_k}$
      - $W_{K_i} \in \mathbb{R}^{D_{model} \times D_k}$
      - $W_{V_i} \in \mathbb{R}^{D_{model} \times D_v}$ - весовые матрицы для $i$-й головы
      - $D_k$ - размерность ключей и запросов
      - $D_v$ - размерность значений
      
      Обычно $D_k = D_v = D_{model} / h$, где $h$ - количество голов.

      С точки зрения линейной алгебры, линейная проекция — это линейное отображение, которое преобразует вектор из одного векторного пространства в другое. В контексте многоголового внимания, входной вектор $X_{embedded}$, принадлежащий $D_{model}$-мерному пространству, подвергается линейным проекциям для создания трех новых векторов: $Q_i$, $K_i$ и $V_i$. Эти векторы находятся в своих собственных подпространствах.

      **Формальное описание:**

      1. **$X_{embedded} \in \mathbb{R}^{D_{model}}$:** Входной вектор в $D_{model}$-мерном пространстве.
      2. **$W_{Q_i} \in \mathbb{R}^{D_{model} \times D_k}$, $W_{K_i} \in \mathbb{R}^{D_{model} \times D_k}$, $W_{V_i} \in \mathbb{R}^{D_{model} \times D_v}$:** Матрицы весов, определяющие линейные отображения.
      3. **Линейные отображения (проекции):**
        - $Q_i = X_{embedded} W_{Q_i}$: Проекция $X_{embedded}$ в $D_k$-мерное подпространство запросов.
        - $K_i = X_{embedded} W_{K_i}$: Проекция $X_{embedded}$ в $D_k$-мерное подпространство ключей.
        - $V_i = X_{embedded} W_{V_i}$: Проекция $X_{embedded}$ в $D_v$-мерное подпространство значений.

      **Зачем это нужно?**

      - **Разделение на подпространства:** Линейные проекции создают отдельные подпространства для запросов, ключей и значений. Это позволяет модели обрабатывать входные данные с разных точек зрения.
      - **Специализация:** Каждое подпространство имеет свою роль: запросы ищут релевантные ключи, а значения используются для агрегации информации.
      - **Обучаемость:** Матрицы весов $W_{Q_i}$, $W_{K_i}$ и $W_{V_i}$ являются обучаемыми параметрами, что позволяет модели адаптироваться к конкретной задаче.
      - **Многоголовость:** Использование нескольких голов (разных наборов матриц) позволяет модели одновременно учитывать разные подпространства, что повышает ее эффективность.

      Таким образом, линейная проекция в механизме многоголового внимания — это способ преобразовать входные данные в различные подпространства, каждое из которых имеет свою роль в процессе обработки информации. Это достигается путем применения линейных отображений, определяемых обучаемыми матрицами весов.

   - **Внимание для каждой головы:** Вычисляется взвешенная сумма значений, где веса определяются функцией softmax от скалярного произведения запросов и ключей:
     $$
     Z_i = \text{Attention}(Q_i, K_i, V_i) = \text{softmax}\left(\frac{Q_i K_i^T}{\sqrt{D_k}}\right) V_i
     $$

   - **Функция Softmax:**

      **Определение:** Функция softmax — это функция, которая преобразует вектор вещественных чисел в вектор вероятностей. Она принимает на вход вектор $z = [z_1, z_2, ..., z_n]$ и возвращает вектор $\sigma(z) = [\sigma(z_1), \sigma(z_2), ..., \sigma(z_n)]$, где каждый элемент $\sigma(z_i)$ вычисляется по формуле:

      $$
      \sigma(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}
      $$

      где:
      - $z_i$ — это $i$-й элемент входного вектора $z$.
      - $e$ — это основание натурального логарифма (приблизительно 2.71828).
      - $n$ — это размерность вектора $z$.

      **Свойства Softmax:**
      - **Нормализация:** Сумма всех элементов выходного вектора $\sigma(z)$ равна 1: $\sum_{i=1}^{n} \sigma(z_i) = 1$.
      - **Вероятности:** Каждый элемент выходного вектора $\sigma(z_i)$ находится в диапазоне от 0 до 1: $0 \leq \sigma(z_i) \leq 1$.
      - **Преобразование:** Softmax преобразует произвольные вещественные числа в вероятности, что делает ее полезной для задач классификации и внимания.

   - **Softmax в Механизме Внимания:**

      В механизме внимания softmax используется для вычисления весов, которые показывают, насколько важен каждый токен при вычислении контекстуализированного представления. В выражении $\text{softmax}\left(\frac{Q_i K_i^T}{\sqrt{D_k}}\right) V_i$:

      - $Q_i$ — это матрица запросов (queries) для $i$-й головы.
      - $K_i$ — это матрица ключей (keys) для $i$-й головы.
      - $V_i$ — это матрица значений (values) для $i$-й головы.
      - $D_k$ — это размерность ключей и запросов.

      **Расчет весов внимания:**
      1. **Скалярное произведение:** Вычисляется скалярное произведение матриц запросов и ключей: $Q_i K_i^T$. Это дает матрицу, где каждый элемент показывает, насколько "совпадает" запрос с ключом.
      2. **Масштабирование:** Результат скалярного произведения делится на $\sqrt{D_k}$. Это масштабирование помогает стабилизировать обучение, предотвращая слишком большие значения, которые могут привести к проблемам с градиентами.
      3. **Softmax:** Функция softmax применяется к результату масштабирования. Это преобразует значения в вероятности, которые представляют собой веса внимания.

   - **Конкатенация голов:** Выходы всех голов конкатенируются:
      $$
      \text{Concat}(Z_1, Z_2, ..., Z_h)
      $$
      где $Z_i = \text{Attention}(Q_i, K_i, V_i)$.
   - **Линейная проекция выхода:** Результат конкатенации проецируется обратно в пространство размерности $D_{model}$:
      $$
      \text{MultiHead}(Q, K, V) = \text{Concat}(Z_1, Z_2, ..., Z_h) W^O
      $$
      где $W^O \in \mathbb{R}^{h D_v \times D_{model}}$ - весовая матрица выходной проекции.

### **Давайте подробно разберем, как работает softmax внутри Multi-Head Attention на вашем примере программного кода ниже.**

Теперь перейдем к самому интересному — Multi-Head Self-Attention, где и используется softmax.

**1. Линейные проекции:**

  - На вход Multi-Head Attention подаются комбинированные эмбеддинги (форма `torch.Size([3, 9, 512])`).
  - Для каждой головы (в вашем примере их 8) входные данные линейно проецируются в три матрицы:
    - **Q (запросы):** `torch.Size([3, 8, 9, 64])`
    - **K (ключи):** `torch.Size([3, 8, 9, 64])`
    - **V (значения):** `torch.Size([3, 8, 9, 64])`
  - Эти проекции выполняются с помощью обучаемых матриц весов $W_Q$, $W_K$ и $W_V$.
  - В вашем примере для первой позиции первого батча (токен `[CLS]`) вы видите примеры матриц Q, K и V.

  **Обратите внимание!:**

```
# Это до подслоя Multi-Head Attention!

Позиция 0 ([CLS]):
Исходный эмбеддинг   X:  [ 0.9007 -2.1055  0.6784]
Позиционное кодир   PE:  [0. 1. 0.]
Комбинированный X + PE:  [ 0.9007 -1.1055  0.6784]

# Это после Multi-Head Attention! 
  Позиция 0 ([CLS]):
    Q: [[ 17.2809  14.7748 -11.5202]
[ -7.0419 -29.7085  54.2687]
[ 29.6011  11.744   -0.8485]
[ -6.7934 -25.4389  67.2095]
[ 28.5684 -16.4333 -13.8622]
[ -4.5139 -61.0905  -0.8532]
[-24.339   -9.4282  -5.367 ]
[  7.8296 -14.4175  16.9908]]
    K: [[  6.637  -10.3847 -29.3882]
[-22.8757 -18.3149 -65.0343]
[ 18.4063  29.4638 -34.1548]
[  5.1229   5.5592  66.0818]
[  9.9801 -20.4229  -7.4216]
[-22.0776   4.2677 -32.6255]
[-40.8423  19.4702   0.3407]
[  8.7071  27.0544 -13.8258]]
    V: [[ 25.8466  12.3776  -7.7585]
[ 32.6146  -2.0634  32.7602]
[ 25.009   11.0889  28.2676]
[ 19.9813 -11.8157  22.7189]
[ 12.4848   6.3136 -28.9884]
[ -1.6635  15.4315 -23.0705]
[ 14.6102  -1.6098 -15.4584]
[ -9.4451 -38.6892  42.4362]]
```

*   **"Q для токена CLS" - это набор векторов в виде матрицы.**
*   **Количество векторов = количеству голов внимания** (в примере 8).
*   **Каждый вектор соответствует отдельной голове внимания.**
*   **Вектор получается линейной проекцией:**  Комбинированный эмбеддинг CLS токена  **умножается на матрицу  `W_q`**  конкретной головы внимания.

**2. Вычисление весов внимания:**

  - Для каждой головы и для каждой позиции в последовательности вычисляются веса внимания.
  - **Скалярное произведение:** Сначала вычисляется скалярное произведение запросов и ключей: $Q_i K_i^T$.
    - В вашем примере, для первой головы и первой позиции (токен `[CLS]`), это будет скалярное произведение вектора Q (размерность 64) на транспонированный вектор K (размерность 64) для каждой позиции в последовательности.
    - Результатом будет матрица размерности (9, 9), где каждый элемент показывает, насколько "совпадает" запрос токена `[CLS]` с ключом каждого из 9 токенов в последовательности.

    <div style="border: 1px solid #000; padding: 10px; margin: 10px;">
    
    **Цель скалярного произведения:**

    Основная цель скалярного произведения на этом этапе — **определить, насколько "совместим" или "релевантен" запрос (Q) одного токена к ключам (K) всех остальных токенов в последовательности.**  В результате мы получаем "сырые" веса внимания, которые потом будут нормализованы Softmax.

    **Что происходит для токена `[CLS]` (первая позиция) и первой головы:**

    1.  **Берем вектор запроса Q для `[CLS]` из первой головы:**
        *   В вашем примере для позиции 0 ([CLS]) и первой головы (первая строка в блоке Q) указан вектор (показаны только первые 3 элемента): `[ 17.2809  14.7748 -11.5202 ...]`.  На самом деле это вектор размерности 64. Обозначим его как  `Q_cls_head1`.

    2.  **Берем векторы ключей K для *всех* позиций (от 0 до 8) из первой головы:**
        *   Для каждой позиции от 0 до 8 в последовательности (токены `[CLS]`, `Всем`, `привет`, `!`, `Я`, `увлекаюсь`, `искусственным`, `интеллектом`, `[SEP]`) есть свой вектор ключей K, полученный из первой головы.  В примере для позиции 0 ([CLS]) и первой головы (первая строка в блоке K) указан вектор: `[  6.637  -10.3847 -29.3882 ...]`.  Обозначим векторы ключей как `K_pos0_head1`, `K_pos1_head1`, `K_pos2_head1`, ..., `K_pos8_head1`. Каждый из них также имеет размерность 64.

    3.  **Вычисляем скалярное произведение между `Q_cls_head1` и каждым вектором ключей `K_pos_j_head1`:**
        *   Для каждой позиции `j` от 0 до 8 мы вычисляем скалярное произведение:
            *   `score_0 = Q_cls_head1 * (K_pos0_head1)^T`  (внимание `[CLS]` на `[CLS]`)
            *   `score_1 = Q_cls_head1 * (K_pos1_head1)^T`  (внимание `[CLS]` на `Всем`)
            *   `score_2 = Q_cls_head1 * (K_pos2_head1)^T`  (внимание `[CLS]` на `привет`)
            *   ...
            *   `score_8 = Q_cls_head1 * (K_pos8_head1)^T`  (внимание `[CLS]` на `[SEP]`)

        *   **Каждое скалярное произведение `score_j` — это одно число (скаляр).** Оно показывает, насколько "совместим" запрос токена `[CLS]` с ключом токена в позиции `j`. Чем больше значение `score_j`, тем больше внимания (пока еще "сырого")  токен `[CLS]` должен уделить токену в позиции `j`.

        В нашем примере для позиции [CLS]:
          ```python
          Q: [[-22.9001 -31.6346  6.0742]    # вектор головы 0
              [ 41.4631   5.2998  6.2346]    # вектор головы 1
              [ 29.6049 -43.8211 -13.9067]   # вектор головы 2
              [ -9.0778  17.0357  -0.9468]   # вектор головы 3
              [ 19.0137  -6.5111 -15.9635]   # вектор головы 4
              [-42.3292 -31.1711  -1.0993]   # вектор головы 5
              [ 22.1916 -19.8376  24.6427]   # вектор головы 6
              [-11.865  -57.7867 -35.5895]]  # вектор головы 7
          ```

          Процесс вычисления:
          
            - Для головы 0:
              * Берем вектор Q головы 0: [-22.9001 -31.6346 6.0742]
              * Умножаем его на все векторы K ТОЛЬКО головы 0
            - Для головы 1:
              * Берем вектор Q головы 1: [41.4631 5.2998 6.2346]
              * Умножаем его на все векторы K ТОЛЬКО головы 1
            - И так далее для каждой головы

    4.  **Результат - ряд скалярных значений:**
        *   В результате этих 9 скалярных произведений мы получаем ряд чисел: `[score_0, score_1, score_2, score_3, score_4, score_5, score_6, score_7, score_8]`.
        *   **Именно этот ряд скалярных значений (после масштабирования и Softmax) станет весами внимания для токена `[CLS]` в первой голове.**  В вашем примере после Softmax эти веса стали `[1. 0. 0. 0. 0. 0. 0. 0. 0.]`.
      </div>

  - **Масштабирование:** Результат скалярного произведения делится на $\sqrt{D_k}$, где $D_k$ — размерность ключей и запросов (в вашем примере $D_k = 64$). Это масштабирование помогает стабилизировать обучение.
  - **Softmax:** Функция softmax применяется к результату масштабирования.
    - Softmax преобразует значения в вероятности, которые представляют собой веса внимания.
    - Softmax применяется к каждой строке матрицы (9, 9), то есть для каждого токена в последовательности вычисляются веса внимания относительно всех остальных токенов.
    - **Важно:** Softmax нормализует веса так, что их сумма равна 1. Это означает, что веса внимания показывают, насколько важен каждый токен при вычислении контекстуализированного представления текущего токена.
    - В вашем примере, для первой головы и первой позиции, вы видите, что веса внимания равны `[1. 0. 0. 0. 0. 0. 0. 0. 0.]`. Это означает, что при вычислении контекстуализированного представления токена `[CLS]` (первая позиция) наибольшее внимание уделяется самому токену `[CLS]`, а остальные токены не имеют значения.

**3. Взвешивание значений:**

  - После вычисления весов внимания, они используются для взвешивания значений (матрицы V).
  - Веса внимания умножаются на соответствующие значения.
  - Результатом является взвешенная сумма значений, которая представляет собой контекстуализированное представление токена.
  - В вашем примере, для первой головы и первой позиции, вы видите матрицу Z, которая является результатом взвешивания значений.

**4. Конкатенация голов:**

  - Выходы всех голов (матрицы Z) конкатенируются по последней размерности.
  - В вашем примере, каждая матрица Z имеет размерность `torch.Size([3, 8, 9, 64])`. После конкатенации получается матрица `torch.Size([3, 9, 512])`.
  - Это означает, что векторы размерности 64 из каждой головы "склеиваются" в один вектор размерности 512.

**5. Линейная проекция выхода:**

  - Результат конкатенации проецируется обратно в пространство размерности $D_{model}$ (512 в вашем примере) с помощью обучаемой матрицы весов $W^O$.
  - Это дает окончательный выход Multi-Head Attention.

    **Назначение матрицы $W^O$:**

    Матрица $W^O$ ("W-output"") выполняет функцию **линейной проекции**, подобно матрицам $W^Q$, $W_K$ и $W_V$, но на этот раз она применяется к **конкатенированному выходу всех голов внимания**.

    **Цель $W^O$**:

    * **Свести размерность:**  После конкатенации выходов всех голов, мы получаем векторное представление, размерность которого увеличена (в вашем примере с 64 до 512, так как 8 голов * 64 размерность каждой головы = 512). Матрица $W^O$ используется для **проекции этого конкатенированного вектора обратно к исходной размерности модели** ($D_{model}$, в вашем примере 512).  Это нужно для того, чтобы выход Multi-Head Attention имел ту же размерность, что и вход, и мог быть интегрирован в остальную часть нейронной сети (например, в слои Feed-Forward Network или Residual Connections).
    * **Смешать информацию от разных голов:**  Конкатенация просто "склеивает" выходы разных голов. Матрица $W^O$ позволяет **взаимодействовать и смешивать информацию, полученную от разных голов внимания**.  Каждая голова могла уловить разные аспекты взаимосвязей в данных, и $W^O$ помогает объединить эти разные "точки зрения" в единое, обобщенное представление.

    **Размерности и перемножение матриц:**

    Давайте рассмотрим размерности матриц, чтобы понять, как происходит перемножение.

    * **Выход каждой головы (Z):**  В вашем примере, после взвешивания значений, каждая голова производит матрицу $Z$ размерности `torch.Size([3, 8, 9, 64])`.  Если мы говорим об **одном батче, одной голове и всех позициях в последовательности**, то это будет матрица размерности `[9, 64]` (последовательность из 9 токенов, вектор размерности 64 для каждого токена).
    * **Конкатенация голов:**  Выходы всех голов (в вашем примере 8 голов) конкатенируются **по последней размерности**.  Это означает, что векторы размерности 64 от каждой головы "складываются" рядом.  Таким образом, для **одного батча и всех позиций в последовательности**, конкатенированный выход будет иметь размерность `[9, 512]` (9 токенов, вектор размерности 512 для каждого токена, где 512 = 8 голов * 64 размерность головы).  Обозначим эту конкатенированную матрицу как $Z_{concat}$.
    * **Матрица $W^O$:** Матрица $W^O$ имеет размерность  $(D_{model} \times D_{model})$, в вашем примере $(512 \times 512)$.  Она преобразует вектор размерности $D_{model}$ (512 после конкатенации) обратно в вектор размерности $D_{model}$ (512).

    **Матричное умножение:**

    Финальный выход Multi-Head Attention получается путем **умножения конкатенированной матрицы $Z_{concat}$ на матрицу $W^O$**:

    $Output_{MHA} = Z_{concat} \times W^O$

    В терминах размерностей:

    `[batch_size, seq_len, num_heads * head_dim]`  умножается на  `[num_heads * head_dim,  D_{model}]`  =  `[batch_size, seq_len, D_{model}]`

    В нашем примере:

    `[3, 9, 512]`  умножается на  `[512, 512]`  =  `[3, 9, 512]`

    > Извиняюсь, не самый удачный пример, так как размерность не изменилась. Вот краткий пример с другой размерностью

    ```
    Вход:  [3, 9, 256]  (D_model = 256)
    |
    |  Multi-Head Attention (num_heads=4, head_dim=128)
    |
    Конкатенация голов: [3, 9, 512]  (размерность увеличилась до 512 = 4 * 128)
    |
    |  Линейная проекция W^O (матрица 512x256)
    |
    Выход: [3, 9, 256]  (размерность вернулась к D_model = 256)
    ```

    **4. Итог по $W^O$:**

    * **$W^O$ - это обучаемая матрица весов.** Она, как и $W^Q$, $W_K$, $W_V$, настраивается в процессе обучения нейронной сети.
    * **$W^O$ применяется после конкатенации выходов всех голов внимания.**
    * **$W^O$ выполняет линейную проекцию, которая:**
        * **Возвращает размерность к исходной $D_{model}$.**
        * **Смешивает информацию, полученную от разных голов внимания.**
    * **Результат умножения на $W^O$ является окончательным выходом слоя Multi-Head Attention.**

**Влияние Softmax:**

- Softmax играет ключевую роль в механизме внимания, преобразуя результаты скалярного произведения запросов и ключей в вероятности.
- Эти вероятности (веса внимания) показывают, насколько важен каждый токен при вычислении контекстуализированного представления текущего токена.
- Softmax обеспечивает, что сумма весов внимания равна 1, что позволяет модели эффективно распределять внимание между различными токенами.

**В итоге:**

Softmax внутри Multi-Head Attention позволяет модели динамически определять, на какие части входной последовательности следует обращать внимание при обработке каждого токена. Это делает модель более гибкой и способной улавливать сложные зависимости в данных.

```python
# Стандартные библиотеки
import math

# Сторонние библиотеки
import numpy as np
import torch


def positional_encoding(max_len: int, d_model: int) -> torch.Tensor:
    """
    Description:
        Генерация позиционных кодирований по формуле из оригинальной статьи Transformer.

    Args:
        max_len: Максимальная длина последовательности.
        d_model: Размерность модели (количество признаков).

    Returns:
        Тензор позиционных кодирований формы (max_len, d_model).

    Examples:
        >>> pe = positional_encoding(10, 512)
        >>> pe.shape
        torch.Size([10, 512])
    """
    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(
        torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
    )

    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe


def print_embeddings(
    tensor: torch.Tensor, tokens: list, title: str, max_elements: int = 3
) -> None:
    """
    Description:
        Визуализация эмбеддингов с метками токенов.

    Args:
        tensor: Тензор эмбеддингов.
        tokens: Список токенов для визуализации.
        title: Заголовок для вывода.
        max_elements: Максимальное количество элементов для отображения.

    Returns:
        None

    Examples:
        >>> embeddings = torch.randn(5, 512)
        >>> tokens = ["token1", "token2", "token3", "token4", "token5"]
        >>> print_embeddings(embeddings, tokens, "Пример эмбеддингов")
    """
    print(f"\n{title}:")
    for idx, (vec, token) in enumerate(zip(tensor, tokens)):
        elements = vec[:max_elements].detach().numpy().round(4)
        print(f"{idx:2d} {token:15}: [{', '.join(f'{x:7.4f}' for x in elements)}...]")


def print_attention_details(
    batch_idx: int,
    head_idx: int,
    pos_idx: int,
    Q: torch.Tensor,
    K: torch.Tensor,
    attention_scores: torch.Tensor,
    attention_weights: torch.Tensor,
    tokens: list,
    num_elements: int = 5,
) -> None:
    """
    Description:
        Выводит детальную информацию о процессе внимания для конкретной позиции.

    Args:
        batch_idx: Индекс батча.
        head_idx: Индекс головы внимания.
        pos_idx: Позиция в последовательности.
        Q: Тензор запросов.
        K: Тензор ключей.
        attention_scores: Тензор сырых оценок внимания.
        attention_weights: Тензор весов внимания после softmax.
        tokens: Список токенов.
        num_elements: Количество элементов для вывода.

    Returns:
        None
    """
    print("\n" + "=" * 60)
    print(
        f"Детали внимания для батча {batch_idx}, головы {head_idx}, "
        f"позиции {pos_idx} ({tokens[batch_idx][pos_idx]}):"
    )

    # Вывод Q вектора
    q_vec = Q[batch_idx, head_idx, pos_idx, :num_elements].detach().numpy()
    print(f"Q вектор (первые {num_elements} элементов):")
    print(f"{q_vec.round(4)}")

    # Вывод K векторов
    print(f"K вектора (первые {num_elements} элементов каждого):")
    for i, token in enumerate(tokens[batch_idx]):
        k_vec = K[batch_idx, head_idx, i, :num_elements].detach().numpy()
        print(f"{i:2d} {token:15}: {k_vec.round(4)}")

    # Ручной расчет скалярных произведений
    manual_scores = []
    q = Q[batch_idx, head_idx, pos_idx]
    for i in range(len(tokens[batch_idx])):
        k = K[batch_idx, head_idx, i]
        score = torch.dot(q, k) / math.sqrt(D_k)
        manual_scores.append(score.item())

    # Получение автоматически рассчитанных оценок
    auto_scores = attention_scores[batch_idx, head_idx, pos_idx].detach().numpy()

    # Сравнение результатов
    print("\nСырые оценки внимания:")
    print(f"Ручной расчет:     {np.array(manual_scores).round(4)}")
    print(f"Автоматический:    {auto_scores.round(4)}")

    # Вывод весов после softmax
    weights = attention_weights[batch_idx, head_idx, pos_idx].detach().numpy()
    print(f"\nВеса внимания после Softmax:")
    print(f"{weights.round(4)}")


# Пример данных
sentences = [
    "Всем привет! Я увлекаюсь искусственным интеллектом",
    "Привет, как дела?",
    "ИИ — это интересно!",
]

# Параметры модели
N = len(sentences)  # Размер батча
L = 9  # Максимальная длина последовательности
D_model = 512  # Размерность эмбеддингов
h = 8  # Количество голов
D_k = D_model // h  # Размерность ключей и запросов
D_v = D_model // h  # Размерность значений

# 1. Токенизация с паддингом
batch_tokens = [
    ["[CLS]", "Всем", "привет", "!", "Я", "увлекаюсь", "искусственным", "интеллектом", "[SEP]"],
    ["[CLS]", "Привет", ",", "как", "дела", "?", "[SEP]", "[PAD]", "[PAD]"],
    ["[CLS]", "ИИ", "—", "это", "интересно", "!", "[SEP]", "[PAD]", "[PAD]"],
]

# 2. Создаем эмбеддинги
embeddings = torch.randn(N, L, D_model)

# 3. Генерируем позиционные кодирования
pe = positional_encoding(L, D_model)

# 4. Комбинируем эмбеддинги с позиционными кодированиями
X_embedded = embeddings + pe  # Broadcasting для батча

# ================= Визуализация процесса =================
print("=" * 60)
print("Шаг 1: Исходные эмбеддинги токенов")
print(f"Форма тензора: {embeddings.shape}")
for batch_idx in range(N):
    print(f"\nБатч {batch_idx + 1}: '{sentences[batch_idx]}'")
    print_embeddings(embeddings[batch_idx], batch_tokens[batch_idx], "Исходные эмбеддинги")

print("\n" + "=" * 60)
print("Шаг 2: Позиционные кодирования")
print(f"Форма тензора: {pe.shape}")
print_embeddings(pe, [f"Позиция {i}" for i in range(L)], "Примеры кодирований")

print("\n" + "=" * 60)
print("Шаг 3: Комбинированные эмбеддинги (X + PE)")
print(f"Форма тензора: {X_embedded.shape}")
for batch_idx in range(N):
    print(f"\nБатч {batch_idx + 1}: '{sentences[batch_idx]}'")
    print_embeddings(X_embedded[batch_idx], batch_tokens[batch_idx], "Результат сложения")

# ================= Подробный вывод для первого батча =================
print("\n" + "=" * 60)
print("Детальный анализ первого батча:")
batch_idx = 0

# Исходные данные
print(f"\nТекст: '{sentences[batch_idx]}'")
print(f"Токены: {batch_tokens[batch_idx]}")

# Сравнение для ключевых позиций
for pos in [0, 2, 4, 6, 8]:
    print(f"\nПозиция {pos} ({batch_tokens[batch_idx][pos]}):")
    print(f"Исходный эмбеддинг:  {embeddings[batch_idx, pos, :3].detach().numpy().round(4)}")
    print(f"Позиционное кодир:   {pe[pos, :3].detach().numpy().round(4)}")
    print(f"Комбинированный:     {X_embedded[batch_idx, pos, :3].detach().numpy().round(4)}")

# ================= Multi-Head Self-Attention =================
print("\n" + "=" * 60)
print("Шаг 4: Multi-Head Self-Attention")

# 1. Линейные проекции
W_Q = torch.randn(h, D_model, D_k)
W_K = torch.randn(h, D_model, D_k)
W_V = torch.randn(h, D_model, D_v)

Q = torch.einsum('nlk,hkd->nhld', X_embedded, W_Q)
K = torch.einsum('nlk,hkd->nhld', X_embedded, W_K)
V = torch.einsum('nlk,hkd->nhld', X_embedded, W_V)

print("\nЛинейные проекции:")
print(f"Форма Q: {Q.shape}")
print(f"Форма K: {K.shape}")
print(f"Форма V: {V.shape}")

# Вывод первых 3 элементов для Q, K, V
for batch_idx in range(N):
    print(f"\nБатч {batch_idx + 1}: '{sentences[batch_idx]}'")
    for pos in [0, 2, 4]:
        print(f"  Позиция {pos} ({batch_tokens[batch_idx][pos]}):")
        print(f"    Q: {Q[batch_idx, :, pos, :3].detach().numpy().round(4)}")
        print(f"    K: {K[batch_idx, :, pos, :3].detach().numpy().round(4)}")
        print(f"    V: {V[batch_idx, :, pos, :3].detach().numpy().round(4)}")

# После расчета attention_weights добавляем:
print_attention_details(
    batch_idx=0,
    head_idx=0,
    pos_idx=0,
    Q=Q,
    K=K,
    attention_scores=attention_scores,
    attention_weights=attention_weights,
    tokens=batch_tokens,
)

# 2. Внимание для каждой головы
attention_scores = torch.einsum('nhld,nhmd->nhlm', Q, K) / math.sqrt(D_k)

# Маскирование паддинга
mask = torch.ones(N, 1, L, L, dtype=torch.bool)
for batch_idx, tokens in enumerate(batch_tokens):
    for i, token in enumerate(tokens):
        if token == "[PAD]":
            mask[batch_idx, :, i:, :] = False
            mask[batch_idx, :, :, i:] = False
attention_scores = attention_scores.masked_fill(~mask, float('-inf'))

attention_weights = torch.softmax(attention_scores, dim=-1)
Z = torch.einsum('nhlm,nhmd->nhld', attention_weights, V)

print("\nВнимание для каждой головы:")
print(f"Форма Z: {Z.shape}")

# Вывод первых 3 элементов для Z
for batch_idx in range(N):
    print(f"\nБатч {batch_idx + 1}: '{sentences[batch_idx]}'")
    for pos in [0, 2, 4]:
        print(f"  Позиция {pos} ({batch_tokens[batch_idx][pos]}):")
        print(f"    Z: {Z[batch_idx, :, pos, :3].detach().numpy().round(4)}")

# Визуализация весов внимания для первой головы и первой позиции
print("\nВизуализация весов внимания для первой головы и первой позиции:")
print(f"Веса внимания (первая голова, первая позиция): {attention_weights[0, 0, 0, :].detach().numpy().round(4)}")

# 3. Конкатенация голов
Z_concat = Z.transpose(1, 2).reshape(N, L, h * D_v)

print("\nКонкатенация голов:")
print(f"Форма Z_concat: {Z_concat.shape}")

# 4. Линейная проекция выхода
W_O = torch.randn(h * D_v, D_model)
multi_head_output = torch.einsum('nlk,kd->nld', Z_concat, W_O)

print("\nЛинейная проекция выхода:")
print(f"Форма MultiHead Output: {multi_head_output.shape}")

# Вывод для первого батча
print("\nДетальный анализ первого батча после Multi-Head Attention:")
batch_idx = 0
for pos in [0, 2, 4, 6, 8]:
    print(f"\nПозиция {pos} ({batch_tokens[batch_idx][pos]}):")
    print(f"Комбинированный:     {X_embedded[batch_idx, pos, :3].detach().numpy().round(4)}")
    print(f"Multi-Head Output:  {multi_head_output[batch_idx, pos, :3].detach().numpy().round(4)}")
```

4. **Слой Add & Norm (после Multi-Head Attention):**
   - **Add (остаточное соединение):** 

        *   **Суть остаточного соединения:**  Остаточное соединение, также известное как skip-connection или residual connection, заключается в том, что **выход слоя Multi-Head Attention добавляется к его исходному входу**.

            В формуле это выглядит так:
            $$
            \text{Output}_{Add1} = X_{embedded} + \text{MultiHead}(Q, K, V)
            $$
            где:
            *   $X_{embedded}$ - это **вход** подслоя Multi-Head Attention. В контексте Transformer, это могут быть эмбеддинги входных токенов, возможно, уже прошедшие через предыдущие слои Transformer.
            *   $\text{MultiHead}(Q, K, V)$ - это **выход** слоя Multi-Head Attention.
            *   $\text{Output}_{Add1}$ - это **результат сложения**, который становится входом для следующего шага - нормализации слоя.

            > То есть, по сути это просто обычное сложение из линейной алгебры двух матриц, а точнее двух тензоров одинаковой размерности.

        *   **Зачем нужны остаточные соединения?**

            *   **Борьба с проблемой затухания градиента:**  В глубоких нейронных сетях, таких как Transformer, градиенты (сигналы для обучения) могут затухать по мере распространения через множество слоев. Остаточные соединения помогают **"перепрыгивать" через слои**, обеспечивая более прямой путь для градиентов. Это облегчает обучение глубоких сетей и позволяет им эффективно учиться.
            *   **Улучшение обучения глубоких сетей:**  Остаточные соединения позволяют обучать **более глубокие и сложные модели**. Без них, добавление новых слоев в глубокую сеть часто не приводит к улучшению производительности, а может даже ухудшить ее. Residual connections позволяют эффективно использовать преимущества глубоких архитектур.
            *   **Сохранение информации о входе:**  Добавляя исходный вход к выходу слоя внимания, мы **сохраняем информацию об исходных эмбеддингах**.  Слой внимания фокусируется на *изменениях* и *уточнении* входных представлений, а остаточное соединение гарантирует, что исходная информация не будет полностью потеряна.

   - **Norm (нормализация слоя):** 

        *   **Суть нормализации слоя:**  Нормализация слоя - это техника нормализации, которая применяется **к выходам нейронного слоя внутри одного обучающего примера**. В отличие от Batch Normalization, которая нормализует по батчу, Layer Normalization нормализует **по признакам внутри одного примера**.

            В Transformer используется именно Layer Normalization. Формула выглядит так:
            $$
            \text{Output}_{Norm1} = \text{LayerNorm}(\text{Output}_{Add1}) = \gamma \frac{\text{Output}_{Add1} - \mu}{\sigma} + \beta
            $$
            где:
            *   $\text{Output}_{Add1}$ - это **вход** для слоя нормализации, который является результатом остаточного соединения.
            *   $\mu$ - это **среднее значение** элементов входа $\text{Output}_{Add1}$ **по размерности признаков** (для каждого примера в отдельности).
            *   $\sigma$ - это **стандартное отклонение** элементов входа $\text{Output}_{Add1}$ **по размерности признаков** (для каждого примера в отдельности).
            *   $\gamma$ (гамма) и $\beta$ (бета) - это **обучаемые параметры масштабирования и сдвига**. Они позволяют сети **настраивать** степень нормализации и восстанавливать оптимальный диапазон значений после нормализации. Изначально $\gamma$ обычно инициализируется единицами, а $\beta$ - нулями.
            *   $\text{Output}_{Norm1}$ - это **выход** слоя нормализации, который становится входом для следующего подслоя (в данном случае, Feed Forward Network).

            > То есть, по сути нормализация слоя, а точнее всех весов выходной матрицы остатков очень похожа на Z-score нормализацию (также известную как стандартизация), которая в статистике используется для преобразования данных к стандартному нормальному распределению. Формула Layer Normalization, которую мы рассматривали:

            $$
            \text{LayerNorm}(x) = \gamma \frac{x - \mu}{\sigma} + \beta
            $$

        *   **Зачем нужна нормализация слоя?**

            *   **Стабилизация обучения:** Нормализация слоя **стабилизирует процесс обучения**, делая его более быстрым и устойчивым. Она помогает **уменьшить внутреннее ковариационное смещение (internal covariate shift)**, то есть изменение распределения входных данных для каждого слоя в процессе обучения. Это происходит потому, что нормализация приводит входные данные к более стандартному диапазону значений (близкому к нулевому среднему и единичному стандартному отклонению).
            *   **Ускорение сходимости:**  Стабилизация обучения позволяет использовать **более высокие скорости обучения (learning rates)** и **ускоряет сходимость** модели к оптимальному решению.
            *   **Улучшение обобщающей способности:**  Нормализация слоя может также способствовать **лучшей обобщающей способности** модели, то есть ее способности хорошо работать на новых, ранее не виденных данных.
            *   **Меньшая зависимость от размера батча:**  В отличие от Batch Normalization, Layer Normalization **не зависит от размера батча**. Это делает ее особенно полезной в ситуациях, когда размер батча небольшой или когда используются рекуррентные нейронные сети, где длина последовательности может варьироваться.

            **Аналогия:** Представьте, что вы настраиваете громкость звука на разных устройствах.  У каждого устройства свой диапазон громкости. Нормализация слоя - это как **приведение громкости к единому стандарту** для всех устройств.  Это облегчает сравнение и обработку звука, делая систему более стабильной и предсказуемой. Параметры $\gamma$ и $\beta$ позволяют немного "подстроить" этот стандарт, чтобы учесть особенности каждого устройства.

5. **Нейронная сеть прямого распространения (Feed Forward Network):**

    **Назначение FFN:**

    FFN - это ключевой компонент в каждом блоке Transformer, который отвечает за **нелинейное преобразование представлений токенов на уровне отдельных позиций**.  В то время как Multi-Head Attention позволяет токенам взаимодействовать друг с другом и учитывать контекст, FFN обрабатывает представление каждого токена **индивидуально**, но уже с учетом контекста, полученного от слоя внимания.

   - На вход подслоя FFN поступает $\text{Output}_{Norm1}$.
   - FFN состоит из двух линейных слоев с функцией активации (например, ReLU, GeLU) между ними:
     $$
     \text{FFN}(\text{Output}_{Norm1}) = \text{Activation}(\text{Output}_{Norm1} W_1 + b_1) W_2 + b_2
     $$
     где:

        - $W_1 \in \mathbb{R}^{D_{model} \times D_{ff}}$
        - $W_2 \in \mathbb{R}^{D_{ff} \times D_{model}}$ - весовые матрицы
        - $b_1 \in \mathbb{R}^{D_{ff}}$, $b_2 \in \mathbb{R}^{D_{model}}$ - векторы смещений
        - $D_{ff}$ - внутренняя размерность FFN (обычно $4 \times D_{model}$).

        **Выход:** FFN преобразует вход и выдает тензор **той же размерности** $(N, L, D_{model})$, где:
            
        - $N$ — размер батча (количество примеров в батче),  
        - $L$ — длина последовательности (число токенов),  
        - $D_{model}$ — скрытая размерность (размер эмбеддингов).  
        
        Этот выход затем передается в следующий слой Transformer или используется для решения задачи (например, классификации или генерации текста).

    **Структура FFN:**

    FFN состоит из **двух последовательных линейных слоев** с **функцией активации** между ними.  Это можно представить как двухслойную полносвязную нейронную сеть, применяемую к каждой позиции в последовательности.

    **Компоненты FFN и формула:**

    $$
    \text{FFN}(x) = \text{Activation}(x W_1 + b_1) W_2 + b_2
    $$

    Разберем каждый компонент формулы:

    1.  **Первый линейный слой (Expansion Layer):**  `(x W_1 + b_1)`
        *   **Вход:**  $x$ - это вход FFN, то есть $\text{Output}_{Norm1}$ размерности `[batch_size, sequence_length, hidden_size]` ($D_{model}$).
        *   **Весовая матрица $W_1$**:  $W_1 \in \mathbb{R}^{D_{model} \times D_{ff}}$ - это **матрица весов первого линейного слоя**.  Она является **обучаемым параметром**.
        *   **Вектор смещения $b_1$**: $b_1 \in \mathbb{R}^{D_{ff}}$ - это **вектор смещения первого линейного слоя**. Он также является **обучаемым параметром**.
        *   **Внутренняя размерность $D_{ff}$**: $D_{ff}$ - это **внутренняя (промежуточная) размерность FFN**.  Обычно она **больше, чем $D_{model}$**, часто в 4 раза больше ($D_{ff} = 4 \times D_{model}$).  Например, если $D_{model} = 512$, то $D_{ff} = 2048$.  **Увеличение размерности** на этом этапе называется **"расширением" (expansion)**.
        *   **Операция:**  Происходит **линейное преобразование** входа $x$ путем матричного умножения на $W_1$ и добавления смещения $b_1$.
        *   **Выход первого линейного слоя:**  Результатом является тензор размерности `[batch_size, sequence_length, D_{ff}]`.  Размерность признакового пространства **увеличилась** с $D_{model}$ до $D_{ff}$.

    2.  **Функция активации (Activation Function):**  `Activation(...)`
        *   **Вход:**  Выход первого линейного слоя размерности `[batch_size, sequence_length, D_{ff}]`.
        *   **Функция активации:**  $\text{Activation}$ - это **нелинейная функция активации**.  В Transformer обычно используются:
            *   **ReLU (Rectified Linear Unit):**  $\text{ReLU}(z) = \max(0, z)$.  Простая и эффективная функция, обнуляющая отрицательные значения.
            *   **GeLU (Gaussian Error Linear Unit):**  Более гладкая функция активации, которая в некоторых случаях показывает лучшие результаты, чем ReLU.  Формула GeLU немного сложнее, но суть в том, что она также вносит нелинейность.
        *   **Назначение функции активации:**  Функция активации **вводит нелинейность** в преобразование.  Без нелинейности, FFN был бы просто еще одним линейным слоем, и Transformer в целом был бы эквивалентен линейной модели, что сильно ограничило бы его выразительность.  Нелинейность позволяет модели учить **сложные, нелинейные зависимости** в данных.
        *   **Выход функции активации:**  Размерность тензора **не меняется** после применения функции активации.  Выход по-прежнему имеет размерность `[batch_size, sequence_length, D_{ff}]`.

    3.  **Второй линейный слой (Contraction Layer):**  `(... ) W_2 + b_2`
        *   **Вход:**  Выход функции активации размерности `[batch_size, sequence_length, D_{ff}]`.
        *   **Весовая матрица $W_2$**:  $W_2 \in \mathbb{R}^{D_{ff} \times D_{model}}$ - это **матрица весов второго линейного слоя**.  Также является **обучаемым параметром**.
        *   **Вектор смещения $b_2$**: $b_2 \in \mathbb{R}^{D_{model}}$ - это **вектор смещения второго линейного слоя**.  Также **обучаемый параметр**.
        *   **Операция:**  Происходит **линейное преобразование** выхода функции активации путем матричного умножения на $W_2$ и добавления смещения $b_2$.
        *   **Выход второго линейного слоя (и FFN в целом):**  Результатом является тензор размерности `[batch_size, sequence_length, D_{model}]`.  Размерность признакового пространства **возвращается** к исходной $D_{model}$.  Это **"сжатие" (contraction)** размерности.

    **Размерности в FFN на примере:**

    Предположим, $D_{model} = 512$ и $D_{ff} = 4 \times D_{model} = 2048$.

    1.  **Вход $x$**:  `[batch_size, sequence_length, 512]`
    2.  **Первый линейный слой $(x W_1 + b_1)$**:
        *   $W_1$ имеет размерность `[512, 2048]`
        *   Выход: `[batch_size, sequence_length, 2048]` (размерность расширилась)
    3.  **Функция активации $\text{Activation}$**:
        *   Вход: `[batch_size, sequence_length, 2048]`
        *   Выход: `[batch_size, sequence_length, 2048]` (размерность не меняется)
    4.  **Второй линейный слой $(... ) W_2 + b_2)$**:
        *   $W_2$ имеет размерность `[2048, 512]`
        *   Выход: `[batch_size, sequence_length, 512]` (размерность сжалась обратно к исходной)

    **Назначение матрицы $W_1$ и $W_2$:**

    *   **$W_1$ (матрица расширения):**  Матрица $W_1$ отвечает за **проекцию входного пространства размерности $D_{model}$ в более широкое пространство размерности $D_{ff}$**.  Это позволяет FFN **увеличить выразительность** и "запомнить" больше информации на промежуточном этапе.
    *   **$W_2$ (матрица сжатия):**  Матрица $W_2$ отвечает за **проекцию обратно из пространства размерности $D_{ff}$ в исходное пространство размерности $D_{model}$**.  Это необходимо, чтобы выход FFN имел ту же размерность, что и вход, и мог быть интегрирован в остальную часть архитектуры Transformer.  Также, матрица $W_2$ позволяет **смешать и агрегировать информацию**, полученную на промежуточном этапе в пространстве большей размерности.

    **Зачем нужен FFN в Transformer?**

    *   **Введение нелинейности:**  FFN вносит **нелинейность** в модель, что критически важно для обучения сложных зависимостей в данных.
    *   **Обработка информации на уровне позиций:**  FFN применяется **независимо к каждой позиции** в последовательности.  Это позволяет модели выполнять **более сложное, нелинейное преобразование** представления каждого токена после того, как контекст был учтен слоем внимания.
    *   **Увеличение выразительности модели:**  За счет расширения размерности до $D_{ff}$ и последующего сжатия обратно до $D_{model}$, FFN позволяет модели **увеличить свою выразительность** и способность к обучению более сложным закономерностям.  Промежуточное пространство большей размерности действует как своего рода "скрытое пространство", где модель может более гибко манипулировать представлениями данных.

```python
# Стандартные библиотеки
import math

# Сторонние библиотеки
import numpy as np
import torch
import torch.nn as nn  # Импортируем модуль nn для LayerNorm


def positional_encoding(max_len: int, d_model: int) -> torch.Tensor:
    """
    Description:
        Генерация позиционных кодирований по формуле из оригинальной статьи Transformer.

    Args:
        max_len: Максимальная длина последовательности.
        d_model: Размерность модели (количество признаков).

    Returns:
        Тензор позиционных кодирований формы (max_len, d_model).

    Examples:
        >>> pe = positional_encoding(10, 512)
        >>> pe.shape
        torch.Size([10, 512])
    """
    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(
        torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
    )

    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe


def print_embeddings(
    tensor: torch.Tensor, tokens: list, title: str, max_elements: int = 3
) -> None:
    """
    Description:
        Визуализация эмбеддингов с метками токенов.

    Args:
        tensor: Тензор эмбеддингов.
        tokens: Список токенов для визуализации.
        title: Заголовок для вывода.
        max_elements: Максимальное количество элементов для отображения.

    Returns:
        None

    Examples:
        >>> embeddings = torch.randn(5, 512)
        >>> tokens = ["token1", "token2", "token3", "token4", "token5"]
        >>> print_embeddings(embeddings, tokens, "Пример эмбеддингов")
    """
    print(f"\n{title}:")
    for idx, (vec, token) in enumerate(zip(tensor, tokens)):
        elements = vec[:max_elements].detach().numpy().round(4)
        print(f"{idx:2d} {token:15}: [{', '.join(f'{x:7.4f}' for x in elements)}...]")


def print_attention_details(
    batch_idx: int,
    head_idx: int,
    pos_idx: int,
    Q: torch.Tensor,
    K: torch.Tensor,
    attention_scores: torch.Tensor,
    attention_weights: torch.Tensor,
    tokens: list,
    num_elements: int = 5,
) -> None:
    """
    Description:
        Выводит детальную информацию о процессе внимания для конкретной позиции.

    Args:
        batch_idx: Индекс батча.
        head_idx: Индекс головы внимания.
        pos_idx: Позиция в последовательности.
        Q: Тензор запросов.
        K: Тензор ключей.
        attention_scores: Тензор сырых оценок внимания.
        attention_weights: Тензор весов внимания после softmax.
        tokens: Список токенов.
        num_elements: Количество элементов для вывода.

    Returns:
        None
    """
    print("\n" + "=" * 60)
    print(
        f"Детали внимания для батча {batch_idx}, головы {head_idx}, "
        f"позиции {pos_idx} ({tokens[batch_idx][pos_idx]}):"
    )

    # Вывод Q вектора
    q_vec = Q[batch_idx, head_idx, pos_idx, :num_elements].detach().numpy()
    print(f"Q вектор (первые {num_elements} элементов):")
    print(f"{q_vec.round(4)}")

    # Вывод K векторов
    print(f"K вектора (первые {num_elements} элементов каждого):")
    for i, token in enumerate(tokens[batch_idx]):
        k_vec = K[batch_idx, head_idx, i, :num_elements].detach().numpy()
        print(f"{i:2d} {token:15}: {k_vec.round(4)}")

    # Ручной расчет скалярных произведений
    manual_scores = []
    q = Q[batch_idx, head_idx, pos_idx]
    for i in range(len(tokens[batch_idx])):
        k = K[batch_idx, head_idx, i]
        score = torch.dot(q, k) / math.sqrt(D_k)
        manual_scores.append(score.item())

    # Получение автоматически рассчитанных оценок
    auto_scores = attention_scores[batch_idx, head_idx, pos_idx].detach().numpy()

    # Сравнение результатов
    print("\nСырые оценки внимания:")
    print(f"Ручной расчет:     {np.array(manual_scores).round(4)}")
    print(f"Автоматический:    {auto_scores.round(4)}")

    # Вывод весов после softmax
    weights = attention_weights[batch_idx, head_idx, pos_idx].detach().numpy()
    print(f"\nВеса внимания после Softmax:")
    print(f"{weights.round(4)}")


# Пример данных
sentences = [
    "Всем привет! Я увлекаюсь искусственным интеллектом",
    "Привет, как дела?",
    "ИИ — это интересно!",
]

# Параметры модели
N = len(sentences)  # Размер батча
L = 9               # Максимальная длина последовательности
D_model = 512       # Размерность эмбеддингов
h = 8               # Количество голов
D_k = D_model // h  # Размерность ключей и запросов
D_v = D_model // h  # Размерность значений
D_ff = 4 * D_model  # Размерность Feed Forward Network

# 1. Токенизация с паддингом
batch_tokens = [
    ["[CLS]", "Всем", "привет", "!", "Я", "увлекаюсь", "искусственным", "интеллектом", "[SEP]"],
    ["[CLS]", "Привет", ",", "как", "дела", "?", "[SEP]", "[PAD]", "[PAD]"],
    ["[CLS]", "ИИ", "—", "это", "интересно", "!", "[SEP]", "[PAD]", "[PAD]"],
]

# 2. Создаем эмбеддинги
embeddings = torch.randn(N, L, D_model)

# 3. Генерируем позиционные кодирования
pe = positional_encoding(L, D_model)

# 4. Комбинируем эмбеддинги с позиционными кодированиями
X_embedded = embeddings + pe  # Broadcasting для батча

# ================= Визуализация процесса =================
print("=" * 60)
print("Шаг 1: Исходные эмбеддинги токенов")
print(f"Форма тензора: {embeddings.shape}")
for batch_idx in range(N):
    print(f"\nБатч {batch_idx + 1}: '{sentences[batch_idx]}'")
    print_embeddings(embeddings[batch_idx], batch_tokens[batch_idx], "Исходные эмбеддинги")

print("\n" + "=" * 60)
print("Шаг 2: Позиционные кодирования")
print(f"Форма тензора: {pe.shape}")
print_embeddings(pe, [f"Позиция {i}" for i in range(L)], "Примеры кодирований")

print("\n" + "=" * 60)
print("Шаг 3: Комбинированные эмбеддинги (X + PE)")
print(f"Форма тензора: {X_embedded.shape}")
for batch_idx in range(N):
    print(f"\nБатч {batch_idx + 1}: '{sentences[batch_idx]}'")
    print_embeddings(X_embedded[batch_idx], batch_tokens[batch_idx], "Результат сложения")

# ================= Подробный вывод для первого батча =================
print("\n" + "=" * 60)
print("Детальный анализ первого батча:")
batch_idx = 0

# Исходные данные
print(f"\nТекст: '{sentences[batch_idx]}'")
print(f"Токены: {batch_tokens[batch_idx]}")

# Сравнение для ключевых позиций
for pos in [0, 2, 4, 6, 8]:
    print(f"\nПозиция {pos} ({batch_tokens[batch_idx][pos]}):")
    print(f"Исходный эмбеддинг:  {embeddings[batch_idx, pos, :3].detach().numpy().round(4)}")
    print(f"Позиционное кодир:   {pe[pos, :3].detach().numpy().round(4)}")
    print(f"Комбинированный:     {X_embedded[batch_idx, pos, :3].detach().numpy().round(4)}")

# ================= Multi-Head Self-Attention =================
print("\n" + "=" * 60)
print("Шаг 4: Multi-Head Self-Attention")

# 1. Линейные проекции
W_Q = torch.randn(h, D_model, D_k)
W_K = torch.randn(h, D_model, D_k)
W_V = torch.randn(h, D_model, D_v)

Q = torch.einsum('nlk,hkd->nhld', X_embedded, W_Q)
K = torch.einsum('nlk,hkd->nhld', X_embedded, W_K)
V = torch.einsum('nlk,hkd->nhld', X_embedded, W_V)

print("\nЛинейные проекции:")
print(f"Форма Q: {Q.shape}")
print(f"Форма K: {K.shape}")
print(f"Форма V: {V.shape}")

# Вывод первых 3 элементов для Q, K, V
for batch_idx in range(N):
    print(f"\nБатч {batch_idx + 1}: '{sentences[batch_idx]}'")
    for pos in [0, 2, 4]:
        print(f"  Позиция {pos} ({batch_tokens[batch_idx][pos]}):")
        print(f"    Q: {Q[batch_idx, :, pos, :3].detach().numpy().round(4)}")
        print(f"    K: {K[batch_idx, :, pos, :3].detach().numpy().round(4)}")
        print(f"    V: {V[batch_idx, :, pos, :3].detach().numpy().round(4)}")

# 2. Внимание для каждой головы
attention_scores = torch.einsum('nhld,nhmd->nhlm', Q, K) / math.sqrt(D_k)

# Маскирование паддинга
mask = torch.ones(N, 1, L, L, dtype=torch.bool)
for batch_idx, tokens in enumerate(batch_tokens):
    for i, token in enumerate(tokens):
        if token == "[PAD]":
            mask[batch_idx, :, i:, :] = False
            mask[batch_idx, :, :, i:] = False
attention_scores = attention_scores.masked_fill(~mask, float('-inf'))

attention_weights = torch.softmax(attention_scores, dim=-1)
Z = torch.einsum('nhlm,nhmd->nhld', attention_weights, V)

print("\nВнимание для каждой головы:")
print(f"Форма Z: {Z.shape}")

# Вывод первых 3 элементов для Z
for batch_idx in range(N):
    print(f"\nБатч {batch_idx + 1}: '{sentences[batch_idx]}'")
    for pos in [0, 2, 4]:
        print(f"  Позиция {pos} ({batch_tokens[batch_idx][pos]}):")
        print(f"    Z: {Z[batch_idx, :, pos, :3].detach().numpy().round(4)}")

# Визуализация весов внимания для первой головы и первой позиции
print("\nВизуализация весов внимания для первой головы и первой позиции:")
print(f"Веса внимания (первая голова, первая позиция): {attention_weights[0, 0, 0, :].detach().numpy().round(4)}")

# 3. Конкатенация голов
Z_concat = Z.transpose(1, 2).reshape(N, L, h * D_v)

print("\nКонкатенация голов:")
print(f"Форма Z_concat: {Z_concat.shape}")

# 4. Линейная проекция выхода
W_O = torch.randn(h * D_v, D_model)
multi_head_output = torch.einsum('nlk,kd->nld', Z_concat, W_O)

print("\nЛинейная проекция выхода:")
print(f"Форма MultiHead Output: {multi_head_output.shape}")

# Вывод для первого батча
print("\nДетальный анализ первого батча после Multi-Head Attention:")
batch_idx = 0
for pos in [0, 2, 4, 6, 8]:
    print(f"\nПозиция {pos} ({batch_tokens[batch_idx][pos]}):")
    print(f"Комбинированный:     {X_embedded[batch_idx, pos, :3].detach().numpy().round(4)}")
    print(f"Multi-Head Output:  {multi_head_output[batch_idx, pos, :3].detach().numpy().round(4)}")

# ================= Add & Norm (после Multi-Head Attention) =================
print("\n" + "=" * 60)
print("Шаг 5: Add & Norm (после Multi-Head Attention)")

# Add (остаточное соединение)
# Выход слоя Multi-Head Attention (multi_head_output) добавляется к входу подслоя Multi-Head Attention (X_embedded)
output_add_norm_1_add = X_embedded + multi_head_output
print("\nAdd (остаточное соединение):")
print(f"Форма Output после Add: {output_add_norm_1_add.shape}")

# Norm (нормализация слоя)
# Применяем Layer Normalization к результату сложения
layer_norm_1 = nn.LayerNorm(D_model) #  D_model - размерность, по которой нормализуем
output_add_norm_1_norm = layer_norm_1(output_add_norm_1_add)
print("\nNorm (нормализация слоя):")
print(f"Форма Output после LayerNorm: {output_add_norm_1_norm.shape}")

# Вывод для первого батча после Add & Norm
print("\nДетальный анализ первого батча после Add & Norm:")
batch_idx = 0
for pos in [0, 2, 4, 6, 8]:
    print(f"\nПозиция {pos} ({batch_tokens[batch_idx][pos]}):")
    print(f"Multi-Head Output:      {multi_head_output[batch_idx, pos, :3].detach().numpy().round(4)}")
    print(f"Output Add:             {output_add_norm_1_add[batch_idx, pos, :3].detach().numpy().round(4)}")
    print(f"Output Add & Norm:      {output_add_norm_1_norm[batch_idx, pos, :3].detach().numpy().round(4)}")


# ================= Feed Forward Network =================
print("\n" + "=" * 60)
print("Шаг 6: Feed Forward Network")

# 1. Первый линейный слой (Expansion Layer)
W_ff_1 = torch.randn(D_model, D_ff)
b_ff_1 = torch.randn(D_ff)
output_ffn_layer_1 = torch.relu(torch.einsum('nlk,kd->nld', output_add_norm_1_norm, W_ff_1) + b_ff_1)
print("\nПервый линейный слой (Expansion Layer):")
print(f"Форма Output после первого линейного слоя: {output_ffn_layer_1.shape}")

# 2. Второй линейный слой (Contraction Layer)
W_ff_2 = torch.randn(D_ff, D_model)
b_ff_2 = torch.randn(D_model)
output_ffn = torch.einsum('nlk,kd->nld', output_ffn_layer_1, W_ff_2) + b_ff_2
print("\nВторой линейный слой (Contraction Layer):")
print(f"Форма Output после второго линейного слоя (FFN Output): {output_ffn.shape}")

# Вывод для первого батча после Feed Forward Network
print("\nДетальный анализ первого батча после Feed Forward Network:")
batch_idx = 0
for pos in [0, 2, 4, 6, 8]:
    print(f"\nПозиция {pos} ({batch_tokens[batch_idx][pos]}):")
    print(f"Output Add & Norm:      {output_add_norm_1_norm[batch_idx, pos, :3].detach().numpy().round(4)}")
    print(f"FFN Output:             {output_ffn[batch_idx, pos, :3].detach().numpy().round(4)}")

# ================= Add & Norm (после Feed Forward Network) =================
print("\n" + "=" * 60)
print("Шаг 7: Add & Norm (после Feed Forward Network)")

# Add (остаточное соединение)
# Выход FFN (output_ffn) добавляется к входу подслоя FFN (output_add_norm_1_norm)
output_add_norm_2_add = output_add_norm_1_norm + output_ffn
print("\nAdd (остаточное соединение после FFN):")
print(f"Форма Output после Add: {output_add_norm_2_add.shape}")

# Norm (нормализация слоя)
# Применяем Layer Normalization к результату сложения
layer_norm_2 = nn.LayerNorm(D_model) #  D_model - размерность, по которой нормализуем
output_add_norm_2_norm = layer_norm_2(output_add_norm_2_add)
print("\nNorm (нормализация слоя после FFN):")
print(f"Форма Output после LayerNorm: {output_add_norm_2_norm.shape}")

# Вывод для первого батча после Add & Norm (после FFN)
print("\nДетальный анализ первого батча после Add & Norm (после FFN):")
batch_idx = 0
for pos in [0, 2, 4, 6, 8]:
    print(f"\nПозиция {pos} ({batch_tokens[batch_idx][pos]}):")
    print(f"FFN Output:             {output_ffn[batch_idx, pos, :3].detach().numpy().round(4)}")
    print(f"Output Add (после FFN):   {output_add_norm_2_add[batch_idx, pos, :3].detach().numpy().round(4)}")
    print(f"Output Add & Norm (после FFN): {output_add_norm_2_norm[batch_idx, pos, :3].detach().numpy().round(4)}")
```

6. **Слой Add & Norm (после Feed Forward):**
   - **Add (остаточное соединение):** Выход FFN добавляется к входу подслоя:
     $$
     \text{Output}_{Add2} = \text{Output}_{Norm1} + \text{FFN}(\text{Output}_{Norm1})
     $$
   - **Norm (нормализация слоя):** Применяется нормализация слоя:
     $$
     \text{Output}_{Norm2} = \text{LayerNorm}(\text{Output}_{Add2})
     $$

7. **Выход кодера:**
   - Выходом каждого слоя кодера является $\text{Output}_{Norm2}$ размерности $(N, L, D_{model})$.
   - После прохождения всех $N_{layers}$ слоев кодера, финальный выход представляет собой матрицу контекстуализированных эмбеддингов размерности $(N, L, D_{model})$.

</details> 

### Подключим необходимые библиотеки

In [ ]:
"""
Данный программный код представляет собой демонстрационный проект по созданию и обучению
модели языкового моделирования на основе архитектуры GPT (Generative Pre-trained Transformer).

Основное функциональное назначение кода:
- Загрузка и предварительная обработка текстовых данных из базы данных SQLite.
- Токенизация текста и создание словаря слов.
- Подготовка наборов данных для обучения и оценки модели.
- Реализация архитектуры GPT, включая блоки трансформера, многоголовое самовнимание
  и слои прямой связи.
- Обучение модели с использованием оптимизатора AdamW и функции потерь CrossEntropyLoss.
- Оценка производительности модели на основе метрик перплексии и потерь.
- Сохранение обученной модели.
- Генерация текстовых последовательностей с помощью обученной модели.

Код является учебным примером и предназначен для демонстрации принципов работы
трансформерных моделей в задачах обработки естественного языка.
"""

# Стандартные библиотеки
import math
import sqlite3

# Сторонние библиотеки для работы с данными и машинным обучением
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Библиотеки для обработки естественного языка
import nltk

# Встроенные типы данных и структуры
from collections import Counter
from typing import List, Optional

# Настройка стиля графиков для лучшей визуализации
seaborn.set(palette='summer')

# Определение устройства для вычислений (GPU если доступен, иначе CPU)
# Почему: Использование GPU значительно ускоряет обучение нейронных сетей.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Загрузка датасета

In [ ]:
# Подключение к базе данных SQLite
conn = sqlite3.connect('../input/wikibooks.sqlite')

# Загрузка данных из базы данных SQLite
df = pd.read_sql_query("SELECT * FROM ru LIMIT 3300", conn)

In [ ]:
# Инициализация списка для хранения предложений
sentences: List[str] = []

# Извлечение и токенизация предложений из текстового корпуса
for sentence_text in tqdm(df['body_text']):
    sentences.extend(
        [x.lower() for x in sent_tokenize(sentence_text, language='russian') if len(x) < 256]
    )

print(f"Количество предложений: {len(sentences)}")

100%|██████████| 3300/3300 [00:10<00:00, 322.19it/s]

Количество предложений 120873


### Train Loop

In [ ]:
# Инициализация счетчика слов
# Для построения словаря необходимо подсчитать частоту каждого слова.
words = Counter()

# Токенизация предложений и подсчет частоты слов
# Создание частотного словаря позволяет определить наиболее распространенные слова
# и сформировать ограниченный словарь для модели.
for sentence_item in tqdm(sentences):
    for word in nltk.word_tokenize(sentence_item):
        words[word] += 1

# Инициализация словаря специальными токенами
# Специальные токены необходимы для обозначения неизвестных слов, начала/конца
# последовательности и заполнения (паддинга) для выравнивания длин предложений.
vocab = {'<unk>', '<bos>', '<eos>', '<pad>'}
VOCAB_SIZE = 20000  # Максимальный размер словаря для обучения модели

# Добавление наиболее часто встречающихся слов в словарь
# Ограничение размера словаря помогает снизить сложность модели и
# уменьшить объем памяти, необходимый для хранения эмбеддингов.
for elem in words.most_common(VOCAB_SIZE):
    vocab.add(elem[0])

print(f"Всего слов в словаре: {len(vocab)}")

100%|██████████| 120873/120873 [00:29<00:00, 4126.04it/s]


Всего слов в словаре: 20004


In [ ]:
# Создание отображений: слово -> индекс и индекс -> слово
# Почему: для работы с нейронными сетями слова должны быть представлены в виде числовых индексов.
word2ind: dict[str, int] = {char: i for i, char in enumerate(vocab)}
ind2word: dict[int, str] = {i: char for char, i in word2ind.items()}

In [ ]:
# Обучает модель на одной эпохе.
def fit_epoch(
    model: nn.Module,
    train_loader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    sheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None
) -> tuple[float, float]:
    """
    Description:
    ---------------
        Проводит одну эпоху обучения модели, вычисляет потери и перплексию.

    Args:
    ---------------
        model: Обучаемая модель нейронной сети.
        train_loader: Загрузчик данных для обучающей выборки.
        criterion: Функция потерь для вычисления ошибки.
        optimizer: Оптимизатор для обновления весов модели.
        sheduler: Опциональный планировщик темпа обучения.

    Returns:
    ---------------
        Кортеж из средней перплексии и средней потери за эпоху.

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> model = GPT(...)
        >>> train_loader = DataLoader(...)
        >>> criterion = nn.CrossEntropyLoss()
        >>> optimizer = torch.optim.AdamW(model.parameters())
        >>> perplexity, loss = fit_epoch(model, train_loader, criterion, optimizer)
        >>> print(f"Perplexity: {perplexity}, Loss: {loss}")
    """
    model.train()  # Переключение модели в режим обучения
    losses: List[float] = []
    perplexity: List[float] = []

    # Итерация по батчам данных из загрузчика
    for batch in train_loader:
        optimizer.zero_grad()   # Обнуление градиентов перед прямым проходом
        # Почему: предотвращает накопление градиентов от предыдущих итераций.

        logits = model(batch['input_ids']) # Прямой проход: получение логитов
        loss = criterion(
            logits, batch['target_ids'].flatten()
        )  # Вычисление потерь
        loss.backward()   # Обратный проход: вычисление градиентов
        optimizer.step()  # Обновление весов модели

        perplexity.append(torch.exp(loss).item())  # Вычисление перплексии
        losses.append(loss.item())                 # Сохранение значения потери

    # Вычисление средней перплексии и потерь за эпоху
    avg_perplexity = sum(perplexity) / len(perplexity)
    avg_losses = sum(losses) / len(losses)
    return avg_perplexity, avg_losses


# Оценивает модель на одной эпохе.
def eval_epoch(
    model: nn.Module,
    val_loader: DataLoader,
    criterion: nn.Module
) -> tuple[float, float]:
    """
    Description:
    ---------------
        Проводит одну эпоху оценки модели на валидационной выборке.

    Args:
    ---------------
        model: Обучаемая модель нейронной сети.
        val_loader: Загрузчик данных для валидационной выборки.
        criterion: Функция потерь для вычисления ошибки.

    Returns:
    ---------------
        Кортеж из средней перплексии и средней потери на валидационной выборке.

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> model = GPT(...)
        >>> val_loader = DataLoader(...)
        >>> criterion = nn.CrossEntropyLoss()
        >>> perplexity, loss = eval_epoch(model, val_loader, criterion)
        >>> print(f"Validation Perplexity: {perplexity}, Validation Loss: {loss}")
    """
    # Переключение модели в режим оценки
    # Почему: отключает Dropout и Batch Normalization для стабильной оценки.
    model.eval()
    perplexity: List[float] = []
    losses: List[float] = []

    # Отключение вычисления градиентов
    # Почему: уменьшает потребление памяти и ускоряет вычисления во время оценки.
    with torch.no_grad():
        for batch in val_loader:
            logits = model(batch['input_ids'])
            loss = criterion(
                logits,
                batch['target_ids'].flatten()
            )
            perplexity.append(torch.exp(loss).item())
            losses.append(loss.item())

    # Вычисление средней перплексии и потерь на валидационной выборке
    avg_perplexity = sum(perplexity) / len(perplexity)
    avg_losses = sum(losses) / len(losses)
    return avg_perplexity, avg_losses


# Обучает модель на заданном количестве эпох.
def train(
    train_dataloader: DataLoader,
    eval_dataloader: DataLoader,
    model: nn.Module,
    epochs: int,
    ignore_index: int = word2ind['<pad>'],
    optimizer: Optional[torch.optim.Optimizer] = None,
    criterion: Optional[nn.Module] = None,
    sheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None
) -> tuple[nn.Module, List[tuple[float, float, float, float]]]:
    """
    Description:
    ---------------
        Функция для обучения модели на заданном количестве эпох.

    Args:
    ---------------
        train_dataloader: Загрузчик данных для обучающей выборки.
        eval_dataloader: Загрузчик данных для валидационной выборки.
        model: Модель нейронной сети для обучения.
        epochs: Количество эпох обучения.
        ignore_index: Индекс, который будет игнорироваться при вычислении потерь
                      (обычно это индекс паддинга).
        optimizer: Опциональный оптимизатор. Если не задан, используется AdamW.
        criterion: Опциональная функция потерь. Если не задана, используется CrossEntropyLoss.
        sheduler: Опциональный планировщик темпа обучения.

    Returns:
    ---------------
        Кортеж из обученной модели и истории обучения (потери и перплексии).

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> model = GPT(...)
        >>> train_dl = DataLoader(...)
        >>> eval_dl = DataLoader(...)
        >>> trained_model, history = train(train_dl, eval_dl, model, 10)
    """
    # Инициализация оптимизатора, если не задан
    if optimizer is None:
        optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

    # Инициализация функции потерь, если не задана
    if criterion is None:
        criterion = nn.CrossEntropyLoss(ignore_index=ignore_index).to(device)

    # Определение параметров для планировщика темпа обучения
    # Почему: планировщик темпа обучения помогает модели сходиться эффективнее,
    # постепенно уменьшая темп обучения.
    MIN_LR = 1e-4
    INITIAL_LR = 3e-4

    lambda_func = lambda epoch: max(0.99 ** epoch, MIN_LR / INITIAL_LR)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_func)

    best_model_wts = model.state_dict()  # Сохранение начальных весов модели
    best_perplexity = float('inf')       # Инициализация лучшей перплексии бесконечностью

    # Список для хранения истории обучения
    history: List[tuple[float, float, float, float]] = []
    log_template = ("\nEpoch {ep:03d} train_loss: {t_loss:0.4f} "
                    "val_loss {v_loss:0.4f} train_perplexity {t_acc:0.4f} "
                    "val_perplexity {v_acc:0.4f}")

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        for epoch in range(epochs):
            # Обучение на одной эпохе
            train_perplexity, train_loss = fit_epoch(model, train_dataloader, criterion, optimizer)
            scheduler.step()  # Обновление темпа обучения

            # Оценка на валидационной выборке
            val_perplexity, val_loss = eval_epoch(model, eval_dataloader, criterion)
            history.append((train_loss, train_perplexity, val_loss, val_perplexity))

            # Сохранение весов лучшей модели
            if val_perplexity < best_perplexity:
                best_perplexity = val_perplexity
                best_model_wts = model.state_dict()

            pbar_outer.update(1)  # Обновление прогресс-бара
            tqdm.write(log_template.format(ep=epoch + 1, t_loss=train_loss,
                                           v_loss=val_loss, t_acc=train_perplexity, v_acc=val_perplexity))

    print(f'Лучшая валидационная перплексия: {best_perplexity:.4f}')
    model.load_state_dict(best_model_wts)  # Загрузка весов лучшей модели

    return model, history

### Функции необходимые при обучении/загрузке датасета/генерации текста

In [ ]:
# Класс для создания датасета слов.
class WordDataset(torch.utils.data.Dataset):
    """
    Description:
    ---------------
        Класс для создания датасета из предложений, преобразуя слова в их числовые индексы.

    Args:
    ---------------
        sentences: Список предложений, каждое из которых является списком токенов.
        word2ind: Словарь для преобразования слов в индексы.

    Attributes:
    ---------------
        data: Список предложений в виде списков индексов.
        word2ind: Словарь для преобразования слов в индексы.
        unk_id: Индекс токена для неизвестных слов.
        bos_id: Индекс токена для начала последовательности.
        eos_id: Индекс токена для конца последовательности.
        pad_id: Индекс токена для заполнения.

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> sentences = [['hello', 'world'], ['foo', 'bar']]
        >>> word2ind = {'hello': 0, 'world': 1, 'foo': 2, 'bar': 3, '<bos>': 4, '<eos>': 5, '<unk>': 6, '<pad>': 7}
        >>> dataset = WordDataset(sentences, word2ind)
        >>> print(len(dataset))
        2
        >>> print(dataset[0])
        [4, 0, 1, 5]
    """

    def __init__(self, sentences: List[List[int]], word2ind: dict[str, int]):
        super().__init__()
        self.data = sentences
        self.word2ind = word2ind
        self.unk_id = self.word2ind['<unk>']
        self.bos_id = self.word2ind['<bos>']
        self.eos_id = self.word2ind['<eos>']
        self.pad_id = self.word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        """
        Description:
        ---------------
            Возвращает токенизированное предложение по индексу, добавляя токены BOS и EOS.

        Args:
        ---------------
            idx: Индекс предложения в датасете.

        Returns:
        ---------------
            Список индексов, представляющих предложение с добавленными токенами BOS и EOS.

        Raises:
        ---------------
            IndexError: Если `idx` выходит за пределы допустимого диапазона.

        Examples:
        ---------------
            >>> dataset = WordDataset([['hello', 'world']], {'hello': 0, 'world': 1, '<bos>': 2, '<eos>': 3, '<unk>': 4, '<pad>': 5})
            >>> dataset[0]
            [2, 0, 1, 3]
        """
        tokenized_sentence = [self.bos_id]
        tokenized_sentence.extend(self.data[idx])  # Добавление слов из предложения
        tokenized_sentence.append(self.eos_id)     # Добавление токена конца последовательности
        return tokenized_sentence

    def __len__(self) -> int:
        """
        Description:
        ---------------
            Возвращает количество предложений в датасете.

        Args:
        ---------------
            Нет.

        Returns:
        ---------------
            Количество предложений.

        Raises:
        ---------------
            Нет явных исключений.

        Examples:
        ---------------
            >>> dataset = WordDataset([['hello', 'world'], ['foo', 'bar']], {})
            >>> len(dataset)
            2
        """
        return len(self.data)


# Выполняет заполнение последовательностей до максимальной длины.
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id: int = word2ind['<pad>'], max_seq_len: int = 96
) -> dict[str, torch.Tensor]:
    """
    Description:
    ---------------
        Функция для объединения батча последовательностей с заполнением (padding)
        до максимальной длины. Обрезает или дополняет последовательности.

    Args:
    ---------------
        input_batch: Список списков индексов, представляющих последовательности.
        pad_id: Индекс токена заполнения.
        max_seq_len: Максимальная длина последовательности после заполнения/обрезки.

    Returns:
    ---------------
        Словарь, содержащий тензоры 'input_ids' и 'target_ids' для модели.

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> batch = [[1, 2, 3], [4, 5]]
        >>> padded_batch = collate_fn_with_padding(batch, pad_id=0, max_seq_len=5)
        >>> print(padded_batch['input_ids'])
        tensor([[1, 2, 3, 0],
                [4, 5, 0, 0]])
        >>> print(padded_batch['target_ids'])
        tensor([[2, 3, 0, 0],
                [5, 0, 0, 0]])
    """
    new_batch: List[List[int]] = []
    for sequence in input_batch:
        if len(sequence) > max_seq_len:
            # Обрезка последовательности, если она длиннее максимальной длины
            # Почему: модели имеют фиксированный размер входной последовательности.
            sequence = sequence[:max_seq_len - 1] + [sequence[-1]]
        else:
            # Добавление паддинга, если последовательность короче максимальной длины
            # Почему: все последовательности в батче должны иметь одинаковую длину.
            for _ in range(max_seq_len - len(sequence)):
                sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    # Разделение на входные и целевые последовательности для обучения
    # Почему: для языкового моделирования входная последовательность предсказывает
    # следующий токен (целевая последовательность).
    processed_batch = {
        'input_ids': sequences[:, :-1],  # Все токены кроме последнего
        'target_ids': sequences[:, 1:]   # Все токены кроме первого
    }

    return processed_batch


# Генерирует текстовую последовательность.
def generate_sequence(
    model: nn.Module,
    dict_2ind: dict[str, int],
    ind2dict: dict[int, str],
    starting_seq: int,
    max_seq_len: int = 256
) -> str:
    """
    Description:
    ---------------
        Генерирует текстовую последовательность, начиная с заданного токена.

    Args:
    ---------------
        model: Обученная модель для генерации.
        dict_2ind: Словарь для преобразования слов в индексы.
        ind2dict: Словарь для преобразования индексов в слова.
        starting_seq: Индекс начального токена для генерации.
        max_seq_len: Максимальная длина генерируемой последовательности.

    Returns:
    ---------------
        Сгенерированная текстовая строка.

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> model = GPT(...) # обученная модель
        >>> word2ind = {'<bos>': 0, 'привет': 1, 'мир': 2, '<eos>': 3}
        >>> ind2word = {0: '<bos>', 1: 'привет', 2: 'мир', 3: '<eos>'}
        >>> generated_text = generate_sequence(model, word2ind, ind2word, word2ind['<bos>'], 10)
        >>> print(generated_text)
        "<bos> привет мир <eos>"
    """
    # Переключение на CPU для генерации, если модель на GPU
    # Почему: Генерация обычно не требует большой вычислительной мощности GPU и может
    # быть выполнена на CPU для удобства.
    current_device = 'cpu'  
    model = model.to(current_device)

    # Инициализация входной последовательности начальным токеном
    idx = torch.zeros((1, 1), dtype=torch.long).to(current_device)
    idx[0, 0] = starting_seq

    # Размер блока для контекста трансформера
    # Почему: трансформеры обрабатывают входные последовательности фиксированного размера.
    BLOCK_SIZE = 256  

    model.eval()                # Переключение модели в режим оценки
    current_len = idx.shape[1]  # Текущая длина последовательности

    # Отключение вычисления градиентов
    with torch.no_grad():
        for _ in range(max_seq_len):
            # Обрезка последовательности до размера блока для контекста
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits = model.forward(idx_cond)                    # Получение логитов
            logits = logits.reshape(1, current_len, -1)         # Изменение формы логитов
            logits = logits[:, -1, :]                           # Выбор логитов для последнего токена
            probs = F.softmax(logits, dim=1)                    # Преобразование логитов в вероятности
            idx_next = torch.multinomial(probs, num_samples=1)  # Выбор следующего токена по вероятностям
            idx = torch.cat((idx, idx_next), dim=1)             # Добавление нового токена к последовательности

            if current_len < BLOCK_SIZE:
                current_len += 1

            if idx_next.item() == dict_2ind['<eos>']:  # Остановка, если сгенерирован токен конца последовательности
                break

    # Преобразование индексов обратно в слова и объединение в строку
    words_sequence = ' '.join([ind2dict[i.item()] for i in idx[0]])

    return words_sequence

### Main Model

In [ ]:
# Реализует блок трансформера.
class TransformerBlock(nn.Module):
    """
    Description:
    ---------------
        Реализует один блок трансформера, включающий многоголовое самовнимание
        и слой прямой связи с нормализацией слоев.

    Args:
    ---------------
        num_heads: Количество голов внимания в блоке MultiHeadSelfAttention.
        n_embed: Размерность эмбеддинга (скрытого слоя).
        block_size: Максимальная длина последовательности, обрабатываемая блоком.

    Attributes:
    ---------------
        mhsa: Модуль MultiHeadSelfAttention.
        feed_forward: Модуль FeedForward.
        norm1: Первый слой нормализации.
        norm2: Второй слой нормализации.

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> block = TransformerBlock(num_heads=4, n_embed=128, block_size=64)
        >>> x = torch.randn(1, 64, 128)
        >>> output = block(x)
        >>> print(output.shape)
        torch.Size([1, 64, 128])
    """

    def __init__(
        self,
        num_heads: int,
        n_embed: int,
        block_size: int
    ):
        super(TransformerBlock, self).__init__()
        # Почему: hidden_dim определяется как n_embed // num_heads, так как каждая голова
        # внимания обрабатывает часть общего эмбеддинга.
        hidden_dim = n_embed // num_heads
        self.mhsa = MultiHeadSelfAttention(num_heads, hidden_dim, n_embed, block_size)
        self.feed_forward = FeedForward(n_embed)
        self.norm1 = nn.LayerNorm(n_embed)  # Нормализация слоя после MHSA
        self.norm2 = nn.LayerNorm(n_embed)  # Нормализация слоя после FeedForward

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Прямой проход блока трансформера.

        Args:
        ---------------
            x: Входной тензор.

        Returns:
        ---------------
            Выходной тензор после прохождения через блок трансформера.

        Raises:
        ---------------
            Нет явных исключений.

        Examples:
        ---------------
            >>> block = TransformerBlock(num_heads=4, n_embed=128, block_size=64)
            >>> x = torch.randn(1, 64, 128)
            >>> output = block(x)
        """
        # Сначала нормализация, затем внимание, затем сложение с исходным входом (residual connection)
        x = x + self.mhsa(self.norm1(x))
        # Снова нормализация, затем FeedForward, затем сложение с результатом предыдущего шага
        x = x + self.feed_forward(self.norm2(x))
        return x


# Реализует слой прямой связи (Feed-Forward Network).
class FeedForward(nn.Module):
    """
    Description:
    ---------------
        Реализует слой прямой связи (Feed-Forward Network) с ReLU активацией
        и Dropout.

    Args:
    ---------------
        n_embed: Размерность входного и выходного эмбеддинга.
        extend_width: Множитель для расширения скрытого слоя (по умолчанию 4).
        dropout: Вероятность дропаута (по умолчанию 0.2).

    Attributes:
    ---------------
        layer: Последовательность слоев: линейный, ReLU, линейный, Dropout.

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> ff = FeedForward(n_embed=128)
        >>> x = torch.randn(1, 64, 128)
        >>> output = ff(x)
        >>> print(output.shape)
        torch.Size([1, 64, 128])
    """

    def __init__(
        self,
        n_embed: int,
        extend_width: int = 4,
        dropout: float = 0.2
    ):
        super(FeedForward, self).__init__()
        # Используется Sequential для объединения нескольких слоев в один модуль.
        self.layer = nn.Sequential(
            nn.Linear(n_embed, extend_width * n_embed),  # Расширяющий линейный слой
            nn.ReLU(),                                   # Активационная функция
            nn.Linear(extend_width * n_embed, n_embed),  # Сжимающий линейный слой
            nn.Dropout(dropout)                          # Слой дропаута для регуляризации
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Прямой проход слоя прямой связи.

        Args:
        ---------------
            x: Входной тензор.

        Returns:
        ---------------
            Выходной тензор после прохождения через слой прямой связи.

        Raises:
        ---------------
            Нет явных исключений.

        Examples:
        ---------------
            >>> ff = FeedForward(n_embed=128)
            >>> x = torch.randn(1, 64, 128)
            >>> output = ff(x)
        """
        return self.layer(x)


# Реализует многоголовое самовнимание.
class MultiHeadSelfAttention(nn.Module):
    """
    Description:
    ---------------
        Реализует механизм многоголового самовнимания (Multi-Head Self-Attention).
        Объединяет выходы нескольких независимых голов внимания.

    Args:
    ---------------
        num_heads: Количество голов внимания.
        hidden_dim: Размерность скрытого состояния для каждой головы.
        n_embed: Размерность входного эмбеддинга.
        block_size: Максимальная длина последовательности.
        dropout: Вероятность дропаута (по умолчанию 0.2).

    Attributes:
    ---------------
        num_heads: Количество голов внимания.
        heads: Список модулей SingleHead.
        project: Линейный слой для проекции объединенных выходов голов.
        drop: Слой дропаута.

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> mhsa = MultiHeadSelfAttention(num_heads=4, hidden_dim=32, n_embed=128, block_size=64)
        >>> x = torch.randn(1, 64, 128)
        >>> output = mhsa(x)
        >>> print(output.shape)
        torch.Size([1, 64, 128])
    """

    def __init__(
        self,
        num_heads: int,
        hidden_dim: int,
        n_embed: int,
        block_size: int,
        dropout: float = 0.2
    ):
        super(MultiHeadSelfAttention, self).__init__()
        self.num_heads = num_heads
        # Создание списка независимых голов внимания
        self.heads = nn.ModuleList(
            [SingleHead(hidden_dim, n_embed, block_size) for _ in range(self.num_heads)]
        )
        self.project = nn.Linear(n_embed, n_embed)  # Проекционный слой для объединения выходов
        self.drop = nn.Dropout(dropout)             # Слой дропаута

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Прямой проход многоголового самовнимания.

        Args:
        ---------------
            x: Входной тензор.

        Returns:
        ---------------
            Выходной тензор после применения многоголового самовнимания.

        Raises:
        ---------------
            Нет явных исключений.

        Examples:
        ---------------
            >>> mhsa = MultiHeadSelfAttention(num_heads=4, hidden_dim=32, n_embed=128, block_size=64)
            >>> x = torch.randn(1, 64, 128)
            >>> output = mhsa(x)
        """
        # Объединение выходов всех голов по последнему измерению
        out = torch.cat([sh(x) for sh in self.heads], dim=-1)
        out = self.project(out)  # Проекция объединенных выходов
        out = self.drop(out)     # Применение дропаута
        return out


# Реализует одну голову самовнимания.
class SingleHead(nn.Module):
    """
    Description:
    ---------------
        Реализует одну голову механизма самовнимания (Self-Attention) с маскированием
        для предотвращения заглядывания в будущее.

    Args:
    ---------------
        hidden_dim: Размерность скрытого состояния для данной головы.
        n_embed: Размерность входного эмбеддинга.
        block_size: Максимальная длина последовательности.
        dropout: Вероятность дропаута (по умолчанию 0.2).

    Attributes:
    ---------------
        key: Линейный слой для вычисления ключей.
        query: Линейный слой для вычисления запросов.
        value: Линейный слой для вычисления значений.
        drop: Слой дропаута.
        tril: Нижнетреугольная матрица для маскирования.

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> sh = SingleHead(hidden_dim=32, n_embed=128, block_size=64)
        >>> x = torch.randn(1, 64, 128)
        >>> output = sh(x)
        >>> print(output.shape)
        torch.Size([1, 64, 32])
    """

    def __init__(
        self,
        hidden_dim: int,
        n_embed: int,
        block_size: int,
        dropout: float = 0.2
    ):
        super(SingleHead, self).__init__()
        # Линейные слои для Q, K, V без смещения, так как оно не требуется для трансформаций
        self.key   = nn.Linear(n_embed, hidden_dim, bias=False)
        self.query = nn.Linear(n_embed, hidden_dim, bias=False)
        self.value = nn.Linear(n_embed, hidden_dim, bias=False)
        self.drop  = nn.Dropout(dropout)
        # Регистрация буфера для нижнетреугольной матрицы
        # Почему: это позволяет избежать заглядывания в будущие токены,
        # что критично для авторегрессивных моделей, таких как GPT.
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Прямой проход одной головы самовнимания.

        Args:
        ---------------
            x: Входной тензор.

        Returns:
        ---------------
            Выходной тензор после применения одной головы самовнимания.

        Raises:
        ---------------
            Нет явных исключений.

        Examples:
        ---------------
            >>> sh = SingleHead(hidden_dim=32, n_embed=128, block_size=64)
            >>> x = torch.randn(1, 64, 128)
            >>> output = sh(x)
        """
        batch_size, sequence_length, channels = x.shape
        k = self.key(x)    # Вычисление ключей   (Batch, T, Hidden_dim)
        q = self.query(x)  # Вычисление запросов (Batch, T, Hidden_dim)

        # Вычисление весов внимания (dot product attention)
        # Почему: нормализация на sqrt(C) стабилизирует градиенты при обучении.
        weights = q @ k.transpose(-2, -1) * channels ** (-0.5)

        # Применение маскирования для предотвращения заглядывания в будущее
        masked_weights = weights.masked_fill(self.tril[:sequence_length, :sequence_length] == 0, float("-inf"))
        masked_probs = F.softmax(masked_weights, dim=-1)  # Применение softmax для получения вероятностей
        masked_probs = self.drop(masked_probs)            # Применение дропаута

        v = self.value(x)       # Вычисление значений (Batch, T, Hidden_dim)
        out = masked_probs @ v  # Умножение весов внимания на значения

        return out


# Реализует модель GPT.
class GPT(nn.Module):
    """
    Description:
    ---------------
        Реализует модель GPT (Generative Pre-trained Transformer) для языкового моделирования.

    Args:
    ---------------
        vocab_size: Размер словаря.
        block_size: Максимальная длина последовательности (контекст).
        n_embed: Размерность эмбеддинга.
        num_heads: Количество голов внимания в каждом блоке трансформера.
        n_layers: Количество трансформерных блоков.

    Attributes:
    ---------------
        vocab_size: Размер словаря.
        block_size: Размер блока.
        embedding: Слой эмбеддинга для токенов.
        positional_embedding_table: Слой эмбеддинга для позиций.
        blocks: Последовательность трансформерных блоков.
        norm: Слой нормализации после блоков.
        fc: Финальный линейный слой для проекции в пространство словаря.

    Raises:
    ---------------
        Нет явных исключений.

    Examples:
    ---------------
        >>> model = GPT(vocab_size=10000, block_size=256, n_embed=384, num_heads=6, n_layers=6)
        >>> x = torch.randint(0, 10000, (1, 64))
        >>> output = model(x)
        >>> print(output.shape)
        torch.Size([64, 10000])
    """

    def __init__(
        self,
        vocab_size: int,
        block_size: int,
        n_embed: int,
        num_heads: int,
        n_layers: int
    ):
        super(GPT, self).__init__()
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.embedding = nn.Embedding(vocab_size, n_embed)  # Слой эмбеддинга токенов

        # Почему: позиционные эмбеддинги необходимы для кодирования информации о порядке токенов,
        # так как механизм внимания не учитывает порядок.
        self.positional_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(
            *[TransformerBlock(num_heads, n_embed, block_size) for _ in range(n_layers)],
        )
        self.norm = nn.LayerNorm(n_embed)         # Финальная нормализация слоя
        self.fc = nn.Linear(n_embed, vocab_size)  # Финальный линейный слой для предсказания токенов

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Прямой проход модели GPT.

        Args:
        ---------------
            x: Входной тензор, содержащий индексы токенов.

        Returns:
        ---------------
            Тензор логитов для каждого токена в словаре.

        Raises:
        ---------------
            Нет явных исключений.

        Examples:
        ---------------
            >>> model = GPT(vocab_size=10000, block_size=256, n_embed=384, num_heads=6, n_layers=6)
            >>> x = torch.randint(0, 10000, (1, 64))
            >>> output = model(x)
        """
        batch_size, sequence_length = x.shape
        token_embeddings = self.embedding(x)  # (Batch, Sequence_Length, Embedding_Dim)
        
        # Создание позиционных индексов для текущей длины последовательности
        positional_embedding = self.positional_embedding_table(
            torch.arange(sequence_length, device=x.device)
        )  # (Sequence_Length, Embedding_Dim)
        
        # Сложение токенных и позиционных эмбеддингов
        token_embeddings = token_embeddings + positional_embedding  # (Batch, Sequence_Length, Embedding_Dim)
        blocks_out = self.blocks(token_embeddings)                  # Проход через блоки трансформера
        blocks_out = self.norm(blocks_out)                          # Нормализация
        logits = self.fc(blocks_out)                                # Проекция в пространство словаря (Batch, Sequence_Length, Vocab_Size)
        
        # Изменение формы логитов для вычисления потерь (Batch * Sequence_Length, Vocab_Size)
        logits = logits.reshape(batch_size * sequence_length, self.vocab_size)
        return logits

### Train

In [ ]:
# Разделение предложений на обучающую и валидационную выборки
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2, random_state=42)

In [ ]:
# Преобразует предложение в последовательность индексов.
def sentence_pre(s: str) -> List[int]:
    """
    Description:
    ---------------
        Преобразует строку предложения в список числовых индексов, используя `word2ind`.
        Неизвестные слова заменяются токеном `<unk>`.

    Args:
    ---------------
        s: Входная строка предложения.

    Returns:
    ---------------
        Список целых чисел, где каждое число - это индекс слова в словаре.
    """
    return [word2ind.get(w, word2ind['<unk>']) for w in nltk.word_tokenize(s)]

In [ ]:
# Применение функции `sentence_pre` к обучающим и валидационным предложениям
train_sentences = list(map(sentence_pre, train_sentences))
eval_sentences  = list(map(sentence_pre, eval_sentences))

In [ ]:
# Создание экземпляров классов WordDataset для обучающей и валидационной выборок
train_dataset = WordDataset(train_sentences, word2ind)
eval_dataset  = WordDataset(eval_sentences,  word2ind)

# Создание DataLoader'ов для обучающей и валидационной выборок
# Почему: DataLoader'ы позволяют эффективно загружать данные батчами,
# применять функцию collate_fn и перемешивать данные.

BATCH_SIZE = 128

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=BATCH_SIZE, shuffle=True, num_workers=0
)
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=BATCH_SIZE, num_workers=0
)

In [ ]:
# Определение гиперпараметров модели GPT
# Почему: эти параметры определяют архитектуру и размер модели.
vocab_size = len(vocab)  # Размер словаря
block_size = 256         # Максимальная длина контекста
n_embed = 384            # Размерность эмбеддинга
num_heads = 6            # Количество голов внимания
n_layers = 6             # Количество слоев трансформера

In [ ]:
# Инициализация модели GPT и перенос её на выбранное устройство (CPU/GPU)
model = GPT(vocab_size=vocab_size, block_size=block_size, n_embed=n_embed, num_heads=num_heads, n_layers=n_layers).to(device)

# Вычисление и вывод количества параметров модели
# Почему: позволяет оценить сложность модели и ее потенциал для переобучения.
num_params = sum(p.numel() for p in model.parameters())
print(model)
print(f"Number of model parameters: {num_params:,}")

GPT(
  (embedding): Embedding(20004, 384)
  (positional_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): TransformerBlock(
      (mhsa): MultiHeadSelfAttention(
        (heads): ModuleList(
          (0-5): 6 x SingleHead(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (drop): Dropout(p=0.2, inplace=False)
          )
        )
        (project): Linear(in_features=384, out_features=384, bias=True)
        (drop): Dropout(p=0.2, inplace=False)
      )
      (feed_forward): FeedForward(
        (layer): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (norm1): LayerNorm((384,), eps=1e-05, elemen

In [ ]:
# Запуск процесса обучения модели
EPOCHS = 30

best_model, losses = train(train_dataloader, eval_dataloader, model, EPOCHS, ignore_index=word2ind["<pad>"])

epoch:   3%|▎         | 1/30 [04:05<1:58:27, 245.07s/it]


Epoch 001 train_loss: 5.7074     val_loss 5.1998 train_perplexirty 389.2737 val_perplexirty 182.2926


epoch:   7%|▋         | 2/30 [08:09<1:54:08, 244.59s/it]


Epoch 002 train_loss: 4.9312     val_loss 4.8145 train_perplexirty 140.1134 val_perplexirty 124.0320


epoch:  10%|█         | 3/30 [12:13<1:49:59, 244.42s/it]

...


Epoch 028 train_loss: 1.7853     val_loss 5.5884 train_perplexirty 5.9880 val_perplexirty 272.1051


epoch:  97%|█████████▋| 29/30 [1:58:00<04:04, 244.13s/it]


Epoch 029 train_loss: 1.7539     val_loss 5.6389 train_perplexirty 5.8000 val_perplexirty 286.3192


epoch: 100%|██████████| 30/30 [2:02:04<00:00, 244.13s/it]


Epoch 030 train_loss: 1.7252     val_loss 5.6779 train_perplexirty 5.6343 val_perplexirty 297.6578
Best val perplexirty: 82.070595


In [ ]:
# Сохранение весов лучшей обученной модели
torch.save(best_model.state_dict(), "best_model.pt")

In [ ]:
# Генерация последовательности слов, начиная с токена "облако"
generate_sequence(best_model, word2ind, ind2word,starting_seq=word2ind['облако'])

'облако 1 ) наличие <unk> в нарушений восприятия ; 2 ) <unk> <unk> в объеме ; 3 ) наличие хронического <unk> крови ; 4 ) увеличение ожидаемой средней продолжительности жизни ; 29 . <eos>'

### Выводы

**Трансформерная архитектура (GPT)**

- В данном пункте необходимо было построить трансформерную архитекутур с нуля. Рассмотрена реализация трансформерной архитектуры Generative Pre-trained Transformer (GPT) с использованием фреймворка PyTorch.

**Реализация**

- В основе архитектуры лежит класс GPT, который представляет собой модульную нейронную сеть с использованием трансформерных блоков. Внутри трансформера используются классы для реализации механизма внимания: SingleHead для одной головы и MultiHeadSelfAttention для организации множественного внимания. Эти классы позволяют модели смотреть "назад" и параллельно обработывать разную информацию.

Также использвались векторные представления слов (embeddings), которые обрабатываются и комбинируются перед подачей в модель

**Обучение модели**

- Модель обучалась на изначальном датасете, с последующим контролем качества на основании функции потерь и перплексии. Используются динамические паддинги для обработки различных длин входных данных. Обучение модели заняло примерно 2 часа, она обучалась 30 эпох.

**Генерация текста**

- После обучения модель способна генерировать текст, начиная с заданного слова. Из-за того, что модель маленькая, было очень сложно сгенерировать целиком осознанный текст. Получались также и осмысленные высказывания.

**Заключение**

- В заключении можно сказать, что трансформерные модели типа GPT демонстрируют более высокую эффективность в задачах генерации текста. Реализация "с нуля" помогает лучше понять механизмы работы таких моделей и дает уникальную возможность адаптации и модификации архитектуры под специфические задачи обработки естественного языка.